# 함수 정의

In [ ]:
import os
import warnings
import numpy as np
import scipy.sparse as sp
import torch
from torch import optim
import torch.nn.functional as F
from sklearn.cluster import SpectralClustering, KMeans
from sklearn.preprocessing import normalize, MinMaxScaler
from sklearn import metrics
import matplotlib.pyplot as plt
from tqdm import tqdm

# 경고 무시 설정
warnings.simplefilter("ignore", category=FutureWarning)
warnings.simplefilter("ignore", category=RuntimeWarning)
warnings.simplefilter("ignore", category=UserWarning)

# 재현성 설정
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

# 사용자 정의 모듈
from model import *
from optimizer import loss_function
from utils import *
from clustering_metric import clustering_metrics

In [ ]:
# 평가지표 정의

import numpy as np
import pandas as pd
from scipy.optimize import linear_sum_assignment
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, 
    confusion_matrix, roc_auc_score, adjusted_rand_score, normalized_mutual_info_score
)

def remap_labels(true_labels, predicted_labels):
    """
    헝가리안 알고리즘을 적용하여 predicted_labels의 ID를 true_labels와 맞춤.
    """
    n_clusters = len(np.unique(true_labels))
    confusion = np.zeros((n_clusters, n_clusters), dtype=int)

    for i in range(len(true_labels)):
        confusion[true_labels[i], predicted_labels[i]] += 1

    row_ind, col_ind = linear_sum_assignment(-confusion)

    remapped_labels = np.zeros_like(predicted_labels)
    for i, j in zip(col_ind, row_ind):
        remapped_labels[predicted_labels == i] = j

    return remapped_labels

def calculate_clustering_metrics(true_labels, predicted_labels):

    # ✅ 1️⃣ 헝가리안 알고리즘 적용하여 클러스터 ID 재매핑
    remapped_labels = remap_labels(true_labels, predicted_labels)
    
    # ✅ 2️⃣ 혼동행렬 생성
    confusion_mat = confusion_matrix(true_labels, remapped_labels)

    # ✅ 3️⃣ Accuracy, Precision, Recall, F1-score 계산
    accuracy = accuracy_score(true_labels, remapped_labels)
    precision = precision_score(true_labels, remapped_labels, average='macro', zero_division=0)
    recall = recall_score(true_labels, remapped_labels, average='macro', zero_division=0)
    f1 = f1_score(true_labels, remapped_labels, average='macro')

    # ✅ 4️⃣ ROC AUC 계산 (다중 클래스 지원)
    try:
        roc_auc = roc_auc_score(true_labels, remapped_labels, multi_class='ovr', average='macro')
    except ValueError:
        roc_auc = 0.0  # ROC AUC가 계산 불가능할 경우 기본값 0.0 설정

    # ✅ 5️⃣ ARI (Adjusted Rand Index) & NMI (Normalized Mutual Information)
    ari = adjusted_rand_score(true_labels, remapped_labels)
    nmi = normalized_mutual_info_score(true_labels, remapped_labels)

    return {
        "Confusion Matrix": confusion_mat,
        "Accuracy": accuracy,
        "Precision": precision,
        "Recall": recall,
        "F1-score": f1,
        "ROC AUC": roc_auc,
        "ARI": ari,
        "NMI": nmi
    }



# VARblockbuster 

In [2]:
import numpy as np
import pandas as pd
import time
import networkx as nx
from sklearn.cluster import KMeans
from statsmodels.tsa.api import VAR
from scipy.sparse import diags
from scipy.stats import multivariate_normal
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, 
    confusion_matrix, roc_auc_score, adjusted_rand_score, normalized_mutual_info_score
)


# ✅ 최종 결과 저장을 위한 리스트
all_results = []


# ✅ (b11, b12) 조합 리스트
b_combinations = [
#    (0.25, 0.01),
    (0.5, 0.01),
#    (0.25, 0.05),
#    (0.5, 0.1), # 추가    
]

# ✅ (b11, b12) 조합 반복
for b11, b12 in b_combinations:
    for node in [30, 50, 100, 150, 200, 300]:
        #######################################
        # 파라미터 설정 (세팅 바꿈)
        n = node  # 총 노드 수 
        k = 2     # 블록 수
        #######################################
    
        block_prob_matrix = [[b11 if i == j else b12 for j in range(k)] for i in range(k)]
        block_sizes = [int(n / k)] * k  # 블록 크기
    
        # ✅ SBM 그래프 생성
        sbm_graph = nx.stochastic_block_model(block_sizes, block_prob_matrix, seed=42)
    
        # ✅ 인접행렬 추출 
        adj_matrix = nx.adjacency_matrix(sbm_graph).toarray()

        # 자기회귀 계수 phi 설정
        phi = 0.99
    
        # ✅ D 행렬 계산 (입차수)
        in_degree = np.sum(adj_matrix, axis=0)
        D_diag = in_degree  # 방향 없는 그래프 → 입차수 사용
        D_inv_sqrt = diags(1.0 / np.sqrt(D_diag + 1e-10), offsets=0)
    
        # ✅ Phi 행렬 계산
        Phi = phi * D_inv_sqrt @ adj_matrix.T @ D_inv_sqrt
        
        
        # 정상성 체크
        # 고유값(Eigenvalues) 계산
        eigenvalues = np.linalg.eigvals(Phi)
    
        # 단위근 여부 확인 후 실행 중단
        if np.any(np.abs(eigenvalues) >= 1):
            print("⚠ 단위근 존재 (비정상적) - 실행을 중단합니다.")
            sys.exit()  # 실행 중단
        else:
            print("✅ 정상적 (Stationary) - 계속 실행 가능")
    
        # ✅ True Labels 생성
        true_labels = []
        for block_idx, block_size in enumerate(block_sizes):
            true_labels.extend([block_idx] * block_size)
    
        for T in [100, 150, 200, 300, 400, 500, 1000]:
            # 평가 지표 저장 리스트 초기화
            accuracy_scores = []
            precision_scores = []
            recall_scores = []
            f1_scores = []
            roc_auc_scores = []
            ari_scores = []
            nmi_scores = []
            total_confusion_matrix = None  
    
            start_time = time.time()  # 실행 시간 측정
            num_iterations = 100  # 반복 횟수
    
            for i in range(num_iterations):
                np.random.seed(i)
    
                # 🔹 VAR(1) 데이터 생성
                Y = np.zeros((T, n))
                Y[0, :] = np.random.normal(size=n)
                epsilon = multivariate_normal.rvs(mean=np.zeros(n), cov=np.eye(n), size=T)
    
                for t in range(1, T):
                    Y[t, :] = Phi @ Y[t - 1, :] + epsilon[t, :]

                # VAR 모델 학습 후 계수 행렬 추정
                model = VAR(Y)
                results = model.fit(1, trend='n')
                
                # 계수 행렬 추정
                coeff_matrix = results.coefs[0]  # [0]은 첫 번째 시차의 계수 행렬
                coeff_matrix = coeff_matrix + coeff_matrix.T
    
                # 🔹 고유값 분해
                eigenvalues, eigenvectors = np.linalg.eigh(coeff_matrix)
                topk_indices = np.argsort(eigenvalues)[-k:]
                topk_eigenvectors = eigenvectors[:, topk_indices]
    
                # 🔹 행별 정규화
                diag = np.sqrt(np.sum(topk_eigenvectors ** 2, axis=1))[:, np.newaxis]
                X = topk_eigenvectors / (diag + 1e-10)
    
                # 🔹 KMeans 클러스터링
                kmeans = KMeans(n_clusters=k, random_state=i)
                cluster_labels = kmeans.fit_predict(X)
    
                # 🔹 클러스터링 성능 평가
                metrics = calculate_clustering_metrics(true_labels, cluster_labels)
    
                end_time = time.time()
    
                # 🔹 평가 지표 저장
                accuracy_scores.append(metrics["Accuracy"])
                precision_scores.append(metrics["Precision"])
                recall_scores.append(metrics["Recall"])
                f1_scores.append(metrics["F1-score"])
                roc_auc_scores.append(metrics["ROC AUC"])
                ari_scores.append(metrics["ARI"])
                nmi_scores.append(metrics["NMI"])
    
            # 🔹 평균 및 표준 편차 계산
            results_summary = {
                'b11':b11, 'b12':b12,
                "Nodes": node,
                "T": T,
                "Accuracy_Mean": np.mean(accuracy_scores),
                "Accuracy_Std": np.std(accuracy_scores),
                "Precision_Mean": np.mean(precision_scores),
                "Precision_Std": np.std(precision_scores),
                "Recall_Mean": np.mean(recall_scores),
                "Recall_Std": np.std(recall_scores),
                "F1-score_Mean": np.mean(f1_scores),
                "F1-score_Std": np.std(f1_scores),
                "ARI_Mean": np.mean(ari_scores),
                "ARI_Std": np.std(ari_scores),
                "NMI_Mean": np.mean(nmi_scores),
                "NMI_Std": np.std(nmi_scores),
                "Simulation_time": end_time - start_time
                
            }
    
            # 🔹 DataFrame에 결과 추가
            all_results.append(results_summary)
    
    
            # 🔹 실행 시간 출력
            print(f"Execution Time for node={node}, T={T}: {end_time - start_time:.5f} sec")

# ✅ 최종 DataFrame 생성
results_df = pd.DataFrame(all_results)


# ✅ 결과 저장 (선택 사항)
#results_df.to_csv(f"블록버스터_시뮬.csv", index=False)

✅ 정상적 (Stationary) - 계속 실행 가능


NameError: name 'calculate_clustering_metrics' is not defined

# AGE

In [5]:
#기존 환경 셋팅
from __future__ import division
from __future__ import print_function
import sys
import argparse
import time
import random
from model import LinTrans, LogReg
from link_pred import * 
import torch


parser = argparse.ArgumentParser()
parser.add_argument('--gnnlayers', type=int, default=3, help="Number of gnn layers")
parser.add_argument('--linlayers', type=int, default=1, help="Number of hidden layers")
parser.add_argument('--epochs', type=int, default=400, help='Number of epochs to train.')
parser.add_argument('--dims', type=int, default=[500], help='Number of units in hidden layer 1.')
parser.add_argument('--lr', type=float, default=0.001, help='Initial learning rate.')

parser.add_argument('--upth_st', type=float, default=0.01, help='Upper Threshold start.') # 상위 0.05 - 고정
parser.add_argument('--lowth_st', type=float, default=0.3, help='Lower Threshold start.') # 하위 0.3 
parser.add_argument('--upth_ed', type=float, default=0.05, help='Upper Threshold end.') # 상위 0.01 - 고정
parser.add_argument('--lowth_ed', type=float, default=0.5, help='Lower Threshold end.') # 하위 0.5 

parser.add_argument('--upd', type=int, default=5, help='Update epoch.')
parser.add_argument('--bs', type=int, default=8, help='Batchsize.') # 16
parser.add_argument('--dataset', type=str, default='citeseer', help='type of dataset.')
parser.add_argument('--no-cuda', action='store_true', default=False,
                    help='Disables CUDA training.')

args = parser.parse_args(args=[])
args.cuda = not args.no_cuda and torch.cuda.is_available()

if args.cuda is True:
    print('Using GPU')
    torch.cuda.manual_seed(SEED)
    os.environ["CUDA_VISIBLE_DEVICES"] = "5"




# 메모리 정리용 함수
def clear_memory():
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    import gc
    gc.collect()


Using GPU
Using GPU


In [6]:
import numpy as np
import torch
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.model_selection import ParameterGrid
from scipy.stats import multivariate_normal

# 필요한 함수는 외부에서 import
# from your_module import LinTrans, clustering, update_similarity, update_threshold

def tune_hyperparams(true_labels, args, T, n, Phi):
    best_score = -np.inf
    best_config = None
    best_model_state = None

    param_grid = {
        "dims": [[d for d in [16, 32, 64, 128, 256, 512] if d < n]],
        "epochs": [30, 35, 40, 45, 50],
        "upth_st": [0.01],
        "lowth_st": [0.3],
        "upth_ed": [0.05, 0.1, 0.15],
        "lowth_ed": [0.5],
        "upd": [5],
        "bs": [8]
    }

    for config in ParameterGrid(param_grid):
        print(config)
        # 설정 적용
        args.upth_st = config['upth_st']
        args.lowth_st = config['lowth_st']
        args.upth_ed = config['upth_ed']
        args.lowth_ed = config['lowth_ed']
        args.upd = config['upd']
        args.bs = config['bs']

        # 데이터 생성
        np.random.seed(42)
        Y = np.zeros((T, n))
        Y[0, :] = np.random.normal(size=n)  
        epsilon = multivariate_normal.rvs(mean=np.zeros(n), cov=np.eye(n), size=T)
        for t in range(1, T):
            Y[t, :] = Phi @ Y[t - 1, :] + epsilon[t, :] 

        features = torch.FloatTensor(MinMaxScaler().fit_transform(Y.T))

        # pos/neg 샘플 정의
        pos_num = 0  # 인접행렬을 안 쓰므로 임의 값
        neg_num = 0  # 마찬가지

        up_eta = (args.upth_ed - args.upth_st) / (config["epochs"] / args.upd)
        low_eta = (args.lowth_ed - args.lowth_st) / (config["epochs"] / args.upd)

        pos_inds, neg_inds = update_similarity(features.numpy(), args.upth_st, args.lowth_st, pos_num, neg_num)
        print(len(pos_inds), len(neg_inds))
        bs = min(args.bs, len(pos_inds))
        length = len(pos_inds)

        dims = [features.shape[1]] + config["dims"]
        model = LinTrans(args.linlayers, dims)
        optimizer = optim.Adam(model.parameters(), lr=0.01)

        if args.cuda:
            model.cuda()
            inx = features.cuda()
        else:
            inx = features

        pos_inds_cuda = torch.LongTensor(pos_inds)
        if args.cuda:
            pos_inds_cuda = pos_inds_cuda.cuda()

        for epoch in range(config['epochs']):
            print(epoch+1)
            st, ed = 0, bs
            model.train()
            length = len(pos_inds)

            while ed <= length:
                sampled_neg = torch.LongTensor(np.random.choice(neg_inds, size=ed - st))
                if args.cuda:
                    sampled_neg = sampled_neg.cuda()
                sampled_inds = torch.cat((pos_inds_cuda[st:ed], sampled_neg), 0)

                optimizer.zero_grad()
                xind = sampled_inds // n
                yind = sampled_inds % n
                x = torch.index_select(inx, 0, xind)
                y = torch.index_select(inx, 0, yind)
                zx = model(x)
                zy = model(y)

                batch_label = torch.cat((torch.ones(ed - st), torch.zeros(ed - st)), dim=0)
                if args.cuda:
                    batch_label = batch_label.cuda()

                batch_pred = model.dcs(zx, zy)
                loss = loss_function(adj_preds=batch_pred, adj_labels=batch_label, n_nodes=ed - st)
                loss.backward()
                optimizer.step()

                st = ed
                ed += bs if ed + bs < length else length - ed

            if (epoch + 1) % args.upd == 0:
                model.eval()
                mu = model(inx)
                hidden_emb = mu.detach().cpu().numpy()
                upth, lowth = update_threshold(upth, lowth, up_eta, low_eta)
                pos_inds, neg_inds = update_similarity(hidden_emb, args.upth_st, args.lowth_st, pos_num, neg_num)
                bs = min(args.bs, len(pos_inds))
                pos_inds_cuda = torch.LongTensor(pos_inds)
                if args.cuda:
                    pos_inds_cuda = pos_inds_cuda.cuda()

                db, nmi, adjscore, acc, f1_macro, precision_macro, recall_macro = clustering(
                    KMeans(n_clusters=2), hidden_emb, true_labels
                )

                if db > best_score:
                    best_score = db
                    best_config = config
                    best_model_state = model.state_dict()

    print(f"\n🎯 Best Config: {best_config}, DBI={-best_score:.4f}")
    return best_config


In [7]:
import warnings
import pandas as pd
import numpy as np
import networkx as nx
import time
from scipy.sparse import diags
from scipy.stats import multivariate_normal
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
import torch
import torch.optim as optim
from tqdm import tqdm
warnings.filterwarnings('ignore')

# 설정
b_combinations = [(0.5, 0.01)]
all_results = []
num_simulations = 100

for b11, b12 in b_combinations:
    for n in [30, 50, 100, 150]:
        k, phi = 2, 0.99
        sbm_graph = nx.stochastic_block_model([n // k] * k,
                    [[b11 if i == j else b12 for j in range(k)] for i in range(k)], seed=42)
        adj_matrix = nx.to_numpy_array(sbm_graph)
        D_diag = np.sum(adj_matrix, axis=0)
        D_inv_sqrt = diags(1.0 / np.sqrt(D_diag + 1e-10))
        Phi = phi * D_inv_sqrt @ adj_matrix @ D_inv_sqrt

        # 단위근 검정
        eigenvalues = np.linalg.eigvals(Phi)
        if np.any(np.abs(eigenvalues) >= 1):
            print("⚠ 단위근 존재 (비정상적) - 실행 중단")
            sys.exit()
        else:
            print("✅ 정상적 (Stationary)")

        True_labels = sum([[i] * (n // k) for i in range(k)], [])

        for T in [100, 150, 200, 300, 400, 500, 1000]:
            start = time.time()

            best_config = tune_hyperparams(True_labels, args, T, n, Phi)

            best_acc, best_nmi, best_adj = 0, 0, 0
            best_f1_macro, best_precision_macro, best_recall_macro = 0, 0, 0
            accuracy_scores, precision_scores, recall_scores, f1_scores, nmi_scores = [], [], [], [], []

            for i in range(1, num_simulations):
                np.random.seed(i)
                Y = np.zeros((T, n))
                Y[0, :] = np.random.normal(size=n)
                epsilon = multivariate_normal.rvs(mean=np.zeros(n), cov=np.eye(n), size=T)
                for t in range(1, T):
                    Y[t, :] = Phi @ Y[t - 1, :] + epsilon[t, :]

                features = torch.FloatTensor(MinMaxScaler().fit_transform(Y.T))

                Cluster = KMeans(n_clusters=k, random_state=i)
                db, nmi, adjscore, acc, f1_macro, precision_macro, recall_macro = clustering(
                    Cluster, features, True_labels
                )
                best_cl = db

                dims = [features.shape[1]] + best_config["dims"]
                model = LinTrans(args.linlayers, dims)
                optimizer = optim.Adam(model.parameters(), lr=0.01)

                if args.cuda:
                    model.cuda()
                    inx = features.cuda()
                else:
                    inx = features

                # 인접행렬 없이 similarity 기반 샘플 생성
                pos_num = n * 5  # 샘플 수 대체값
                neg_num = n * n - pos_num
                up_eta = (best_config["upth_ed"] - best_config["upth_st"]) / (best_config["epochs"]/best_config["upd"])
                low_eta = (best_config["lowth_ed"] - best_config["lowth_st"]) / (best_config["epochs"]/best_config["upd"])

                pos_inds, neg_inds = update_similarity(features.numpy(), best_config["upth_st"],
                                                       best_config["lowth_st"], pos_num, neg_num)
                upth, lowth = update_threshold(best_config["upth_st"], best_config["lowth_st"], up_eta, low_eta)

                bs = min(best_config["bs"], len(pos_inds))
                pos_inds_cuda = torch.LongTensor(pos_inds).cuda()

                for epoch in tqdm(range(best_config["epochs"])):
                    st, ed = 0, bs
                    epoch_loss = 0
                    model.train()
                    length = len(pos_inds)

                    while ed <= length:
                        sampled_neg = torch.LongTensor(np.random.choice(neg_inds, size=ed-st)).cuda()
                        sampled_inds = torch.cat((pos_inds_cuda[st:ed], sampled_neg), 0)
                        optimizer.zero_grad()
                        xind = sampled_inds // n
                        yind = sampled_inds % n
                        x = torch.index_select(inx, 0, xind)
                        y = torch.index_select(inx, 0, yind)
                        zx = model(x)
                        zy = model(y)
                        batch_label = torch.cat((torch.ones(ed-st), torch.zeros(ed-st)), dim=0).cuda()
                        batch_pred = model.dcs(zx, zy)
                        loss = loss_function(adj_preds=batch_pred, adj_labels=batch_label, n_nodes=ed-st)
                        loss.backward()
                        optimizer.step()
                        epoch_loss += loss.item()
                        st = ed
                        ed += bs if ed + bs < length else length - ed
                        
                        batch_num+=1

                    if (epoch + 1) % best_config["upd"] == 0:
                        model.eval()
                        mu = model(inx)
                        hidden_emb = mu.cpu().data.numpy()
                        upth, lowth = update_threshold(upth, lowth, up_eta, low_eta)
                        pos_inds, neg_inds = update_similarity(hidden_emb, upth, lowth, pos_num, neg_num)
                        bs = min(best_config["bs"], len(pos_inds))
                        pos_inds_cuda = torch.LongTensor(pos_inds).cuda()
                        db, nmi, adjscore, acc, f1_macro, precision_macro, recall_macro = clustering(
                            Cluster, hidden_emb, True_labels
                        )
                        if db >= best_cl:
                            best_emb = hidden_emb
                            best_cl = db
                            best_acc, best_nmi, best_adj = acc, nmi, adjscore
                            best_f1_macro, best_precision_macro, best_recall_macro = f1_macro, precision_macro, recall_macro

                # 저장
                accuracy_scores.append(best_acc)
                recall_scores.append(best_recall_macro)
                precision_scores.append(best_precision_macro)
                f1_scores.append(best_f1_macro)
                nmi_scores.append(best_nmi)

            end_time = time.time()
            print(f"Execution Time for {b11,b12} - node={n}, T={T}: {end_time - start:.2f}s")

            results_summary = {
                'b11': b11, 'b12': b12, "Nodes": n, "T": T,
                "epoch": best_config['epochs'], "dims": best_config['dims'],
                "Accuracy_Mean": np.mean(accuracy_scores),
                "Accuracy_Std": np.std(accuracy_scores),
                "Precision_Mean": np.mean(precision_scores),
                "Precision_Std": np.std(precision_scores),
                "Recall_Mean": np.mean(recall_scores),
                "Recall_Std": np.std(recall_scores),
                "F1-score_Mean": np.mean(f1_scores),
                "F1-score_Std": np.std(f1_scores),
                "NMI_Mean": np.mean(nmi_scores),
                "NMI_Std": np.std(nmi_scores),
                'Simulation_Time': end_time - start
            }
            print(results_summary)
            all_results.append(results_summary)

df_results = pd.DataFrame(all_results)
df_results.to_csv("AGE_simulation.csv", index=False, encoding="utf-8-sig")
print("📁 'AGE_simulation.csv' 저장 완료")

✅ 정상적 (Stationary)
{'bs': 8, 'dims': [16], 'epochs': 30, 'lowth_ed': 0.5, 'lowth_st': 0.3, 'upd': 5, 'upth_ed': 0.05, 'upth_st': 0.01}
9 630
1


KeyboardInterrupt: 

In [8]:
# hyperparam_tuning_fit.py

import numpy as np
import scipy.sparse as sp
import torch
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.model_selection import ParameterGrid


# 필요한 사용자 정의 함수들은 외부에서 import 하거나 별도 구현 필요
# from your_module import LinTrans, preprocess_graph, clustering, update_similarity, update_threshold

def tune_hyperparams(true_labels, args, T, n, Phi):
    best_score = -np.inf
    best_config = None
    best_model_state = None
    
    dim_candidates = [d for d in [ 32, 64, 128] if d < 0.8*T] #16, 32, 
    if not dim_candidates:
        print(f"T={T}보다 작은 dims 없음 → 이 경우 건너뜀")
        return None  # 또는 적절한 처리

    param_grid = {
        "dims": [[d] for d in dim_candidates],  # ✅ 하나짜리 리스트로 포장        
        "threshold": [0], #, 85, 90
        "epochs": [30, 35, 40, 45, 50],
        "upth_st": [0.1], #, 0.03, 0.05
        "lowth_st": [0.3],
        "upth_ed": [0.05, 0.03, 0.01],
        "lowth_ed": [0.5],
        "upd": [5],
        "bs": [8]
    }

    for config in ParameterGrid(param_grid):
#        print(f"⏳ Trying config: {config}")
        args.upth_st = config['upth_st']
        args.lowth_st = config['lowth_st']
        args.upth_ed = config['upth_ed']
        args.lowth_ed = config['lowth_ed']
        args.upd = config['upd']
        args.bs = config['bs']

        np.random.seed(42)  # 시드 고정
        train_size = int(T * 0.8)  # 시뮬레이션 시작 전 train 길이 설정 
        Y = np.zeros((T, n))
        Y[0, :] = np.random.normal(size=n)  
        epsilon = multivariate_normal.rvs(mean=np.zeros(n), cov=np.eye(n), size=T)
        for t in range(1, T):
            Y[t, :] = Phi @ Y[t - 1, :] + epsilon[t, :] # Y[t - 1, :]가 t-1행을 열벡터로 표현하여 계수행렬 Phi와 곱해짐 , 행이 위로 갈수록 과거 
        
        # 전체 데이터로 인접행렬 추정 + 라벨 
#        corr_matrix = (np.corrcoef(Y.T)) 
#        np.fill_diagonal(corr_matrix, 0)  # self-loop 제거 
#        threshold = np.percentile(corr_matrix, config["threshold"]) 
#        precision_matrix = (corr_matrix > threshold).astype(int)

#        precision_matrix = (np.corrcoef(Y.T))

        
#        adj = np.array(np.where(precision_matrix > 0)).T  # A 원소가 1인 순서쌍 출력 ([[1 0] [1 1] ...] 인 array 형태) 
#        adj = sp.csr_matrix((np.ones(len(adj)), (adj[:,0], adj[:,1])), shape=(len(true_labels), len(true_labels))) # 추정된 A 
        
        # Train/Test 분리 (8:2) 후 사이즈 맞추기 
        train_data, test_data = Y[:train_size], Y[train_size:] 
        train_len, feature_dim = train_data.shape

        features = torch.FloatTensor(MinMaxScaler().fit_transform(train_data.T))
#        adj = adj - sp.dia_matrix((adj.diagonal()[np.newaxis, :], [0]), shape=adj.shape)
#        adj.eliminate_zeros()
        # 스무딩 후
#        adj_1st = (adj + sp.eye(n)).toarray()
#        adj_label = torch.FloatTensor(adj_1st).reshape([-1,])

        pos_num = 0 #len(adj.indices) 
        neg_num = 0 #n * n - pos_num

        up_eta = (args.upth_ed - args.upth_st) / (config["epochs"] / args.upd)
        low_eta = (args.lowth_ed - args.lowth_st) / (config["epochs"] / args.upd)

        pos_inds, neg_inds = update_similarity(features.numpy(), args.upth_st, args.lowth_st, pos_num, neg_num) # 
#        pos_inds, neg_inds = update_similarity_1(precision_matrix, args.upth_st, args.lowth_st, pos_num, neg_num) 
        upth, lowth = update_threshold(args.upth_st, args.lowth_st, up_eta, low_eta) 

        bs = min(args.bs, len(pos_inds))
        length = len(pos_inds)

        dims = [features.shape[1]] + config["dims"]
        model = LinTrans(args.linlayers, dims)
#        print(model)
        optimizer = optim.Adam(model.parameters(), lr=0.01)

        if args.cuda:
            model.cuda()
            inx = features.cuda()
#            adj_label = adj_label.cuda()
        else:
            inx = features

        pos_inds_cuda = torch.LongTensor(pos_inds)
        if args.cuda:
            pos_inds_cuda = pos_inds_cuda.cuda()

#        print('튜닝을 위한 Start Training...')
        
        for epoch in range(config['epochs']):
#            print(f'{epoch+1}th , pos_length = {length}')
            st, ed = 0, bs # bs가 len(pos_inds) ed 
            batch_num = 0
            model.train()
            length = len(pos_inds)
            while ed <= length:
#                print(f'{epoch} 번째 에폭 , ed :{ed} , length: {ed - st}')
                sampled_neg = torch.LongTensor(np.random.choice(neg_inds, size=ed-st))
                if args.cuda:
                    sampled_neg = sampled_neg.cuda()
                sampled_inds = torch.cat((pos_inds_cuda[st:ed], sampled_neg), 0)

                optimizer.zero_grad()
                xind = sampled_inds // n
                yind = sampled_inds % n
                x = torch.index_select(inx, 0, xind) #x에 해당하는 특성 벡터 
                y = torch.index_select(inx, 0, yind) #y에 해당하는 특성 벡터 
                zx = model(x)
                zy = model(y)
                batch_label = torch.cat((torch.ones(ed-st), torch.zeros(ed-st)), dim=0).cuda() # 라벨 생성 
                if args.cuda:
                    batch_label = batch_label.cuda()
                batch_pred = model.dcs(zx, zy)
                loss = loss_function(adj_preds=batch_pred, adj_labels=batch_label, n_nodes=ed-st)
                loss.backward()
                optimizer.step()

                st = ed
                if ed < length and ed + bs >= length: 
                    ed += length - ed
                else:
                    ed += bs

                batch_num+=1

            if (epoch + 1) % args.upd == 0:
                model.eval()
                mu = model(inx)
                hidden_emb = mu.detach().cpu().numpy()
                upth, lowth = update_threshold(upth, lowth, up_eta, low_eta)
#                precision_matrix = (np.corrcoef(hidden_emb))             
                pos_inds, neg_inds = update_similarity(hidden_emb, args.upth_st, args.lowth_st, pos_num, neg_num) 
                bs = min(args.bs, len(pos_inds))
                pos_inds_cuda = torch.LongTensor(pos_inds)
                if args.cuda:
                    pos_inds_cuda = pos_inds_cuda.cuda()

                db, nmi, adjscore, acc, f1_macro, precision_macro, recall_macro = clustering(KMeans(n_clusters=2), hidden_emb, True_labels)

                if db > best_score:
                    best_score = db
                    best_config = config
                    best_model_state = model.state_dict()
                    
#            print(f'배치 회수: {batch_num}') 
            
                    

    print(f"\n🎯 Best Config: {best_config}, DBI={-best_score:.4f}")
    return best_config

In [9]:
import warnings
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from scipy.sparse import diags
from scipy.stats import multivariate_normal
import time 
from sklearn.metrics.pairwise import cosine_similarity
warnings.filterwarnings('ignore')


# ✅ (b11, b12) 조합 리스트
b_combinations = [
#    (0.25, 0.01),
    (0.5, 0.01),
#    (0.25, 0.05),
#    (0.5, 0.1),
]

all_results = []
num_simulations = 100


for b11, b12 in b_combinations:
    for n in [150, 100, 50, 30 ]: #  
        # 파라미터 설정 
        k, phi = 2, 0.99        
        # SBM을 이용한 계수행렬 생성
        sbm_graph = nx.stochastic_block_model([n // k] * k, [[b11 if i == j else b12 for j in range(k)] for i in range(k)], seed=42)
        adj_matrix = nx.to_numpy_array(sbm_graph)
        D_diag = np.sum(adj_matrix, axis=0)  
        D_inv_sqrt = diags(1.0 / np.sqrt(D_diag + 1e-10))
        Phi = phi * D_inv_sqrt @ adj_matrix @ D_inv_sqrt # Φ 생성 

        # 단위근 검정 
        eigenvalues = np.linalg.eigvals(Phi)
        if np.any(np.abs(eigenvalues) >= 1):
            print("⚠ 단위근 존재 (비정상적) - 실행을 중단합니다.")
            sys.exit()  # 실행 중단
        else:
            print("✅ 정상적 (Stationary) - 계속 실행 가능")

        # True label 생성 (test 성능 평가를 위해 쓸 것) 
        True_labels = sum([[i] * (n // k) for i in range(k)], [])
        
        for T in [100, 150, 200, 300, 400, 500, 1000]:

            start = time.time() # 시뮬레이션 시작 타임스탬프

            best_config= tune_hyperparams(True_labels, args, T, n, Phi)
            
            # best_ 성능 저장 초기화
            best_acc, best_nmi, best_adj = 0, 0, 0
            best_f1_macro, best_precision_macro, best_recall_macro = 0, 0, 0
            accuracy_scores = []
            precision_scores = []
            recall_scores = []
            f1_scores = []                #ari_scores = []
            nmi_scores = []
            
#            print('best params로 fitting')

            for i in range(1, num_simulations) :

                np.random.seed(i)                
                Y = np.zeros((T, n))
                Y[0, :] = np.random.normal(size=n)  
                epsilon = multivariate_normal.rvs(mean=np.zeros(n), cov=np.eye(n), size=T)
                for t in range(1, T):
                    Y[t, :] = Phi @ Y[t - 1, :] + epsilon[t, :] 
                              
#                corr_matrix = (np.corrcoef(Y.T))
#                np.fill_diagonal(corr_matrix, 0)  # self-loop 제거 
#                threshold = np.percentile(corr_matrix, best_config["threshold"])
#                precision_matrix = (corr_matrix > threshold).astype(int)
#                adj = np.array(np.where(precision_matrix > 0)).T  # A 원소가 1인 순서쌍 출력 ([[1 0] [1 1] ...] 인 array 형태) 
#                adj = np.array(adj)

                label = True_labels
#                adj = sp.csr_matrix((np.ones(len(adj)), (adj[:,0], adj[:,1])), shape=(len(label), len(label)))
                
                features = torch.FloatTensor(MinMaxScaler().fit_transform(Y.T))
                    
                Cluster = KMeans(n_clusters=k, random_state=i)
                db, nmi, adjscore, acc, f1_macro, precision_macro, recall_macro = clustering(Cluster, features, True_labels) 
                best_cl = db
#                adj_1st = (adj + sp.eye(n)).toarray()
#                adj_label = torch.FloatTensor(adj_1st)
#                adj_label = adj_label.reshape([-1,]) #1차원 벡터로 펴줌

                # 모델 정의
                dims = [features.shape[1]] + best_config["dims"]
                model = LinTrans(args.linlayers, dims)
                optimizer = optim.Adam(model.parameters(), lr=0.01) # Adam으로 최적화
                if args.cuda:
                    model.cuda()
                    inx = features.cuda() # 스무딩된 행렬 X틸다를 행 단위로 array 
#                    adj_label = adj_label.cuda()

                pos_num = 0 # 양수 샘플 개수 정의 
                n_nodes = n
                neg_num = 0 # 음수 샘플 개수 정의 
            
                up_eta = (best_config["upth_ed"] - best_config["upth_st"]) / (best_config["epochs"]/best_config["upd"]) # 몇 번의 업데이트에서 변화(eta)를 적용할 것인지 계산
                low_eta = (best_config["lowth_ed"] - best_config["lowth_st"]) / (best_config["epochs"]/best_config["upd"]) # 몇 번의 업데이트에서 변화(eta)를 적용할 것인지 계산
            
                #처음엔 인접행렬에서 pos_num , neg_num 정의 
                pos_inds, neg_inds = update_similarity(normalize(features.numpy()), best_config["upth_st"], best_config["lowth_st"], pos_num, neg_num) 
                upth, lowth = update_threshold(best_config["upth_st"], best_config["lowth_st"], up_eta, low_eta) # 임계값 업데이트 
            
                bs = min(best_config["bs"], len(pos_inds)) # 배치 사이즈 : bs
                length = len(pos_inds)
                
                pos_inds_cuda = torch.LongTensor(pos_inds).cuda()
                for epoch in (range(best_config["epochs"])):
#                    print(epoch+1)
                    
                    st, ed = 0, bs # bs가 len(pos_inds) ed 
                    batch_num = 0
                    model.train()
                    length = len(pos_inds)
                    epoch_loss = 0
                    
                    while ( ed <= length ): # 배치 사이즈가 pos 길이보다 작거나 같은 동안 
                        sampled_neg = torch.LongTensor(np.random.choice(neg_inds, size=ed-st)).cuda() # neg_inds에서 ed-st 크기 랜덤 샘플하여 sampled_neg 
                        sampled_inds = torch.cat((pos_inds_cuda[st:ed], sampled_neg), 0) # 현재 배치에서 생성된 양성 샘플과 음성 샘플 합체 
                        optimizer.zero_grad() # 그래디언트 초기화 
                        xind = sampled_inds // n_nodes
                        yind = sampled_inds % n_nodes
                        x = torch.index_select(inx, 0, xind) # 샘플링된 노드 쌍에서 첫번째 노드의 인덱스 추출 
                        y = torch.index_select(inx, 0, yind) # 샘플링된 노드 쌍에서 두번째 노드의 인덱스 추출  - 각각의 노드 feature가 됨 
                        zx = model(x) # 모델을 거친 후의 노드 임베딩 
                        zy = model(y) # 모델을 거친 후의 노드 임베딩 
                        batch_label = torch.cat((torch.ones(ed-st), torch.zeros(ed-st)), dim=0).cuda() # 라벨 생성 
                        batch_pred = model.dcs(zx, zy) #  두 노드 간 유사도 계산 (link_pred) 
                        loss = loss_function(adj_preds=batch_pred, adj_labels=batch_label, n_nodes=ed-st) # 예측한 링크 확률과 실제 정답 간 차이, 최소화하도록 모델 학습 
                        
                        loss.backward() # 손실 기반 그래디언트 계산 
                        cur_loss = loss.item() # 현재 배치의 손실값 저장 
                        optimizer.step() # 모델 가중치 업데이트 
                        epoch_loss += loss.item()  # 손실 누적
                        
                        st = ed # 다음 배치 시작 위치 업데이트 
                        batch_num += 1
                        if ed < length and ed + bs >= length: 
                            ed += length - ed
                        else:
                            ed += bs
                
                    if (epoch + 1) % best_config["upd"] == 0: # 업데이트 횟수 T의 배수번째인 에폭마다 전체 데이터
                        model.eval() # 평가 모드로 전환 
                        mu = model(inx) #전체 feature_matrix의 행을 변환 
                        hidden_emb = mu.cpu().data.numpy()
                        upth, lowth = update_threshold(upth, lowth, up_eta, low_eta) # threshold 업데이트하여 threshold 새로 정의  
                        pos_inds, neg_inds = update_similarity(hidden_emb, upth, lowth, pos_num, neg_num) # 새로 정의된 threshold로 pos neg 샘플 인덱스 (imbalanced)
                        bs = min(best_config["bs"], len(pos_inds)) #bs 새로 정의 - pos_num 새로 정의돼서 
                        pos_inds_cuda = torch.LongTensor(pos_inds).cuda()
                        db, nmi, adjscore, acc, f1_macro, precision_macro, recall_macro = clustering(Cluster, hidden_emb, True_labels) # 지금까지 학습된 모델로 X틸다 확장시킨 행렬로 클러스터링
                        if db >= best_cl:
                            best_cl = db
                            best_acc, best_nmi, best_adj = acc, nmi, adjscore
                            best_f1_macro, best_precision_macro, best_recall_macro = f1_macro, precision_macro, recall_macro
#                    tqdm.write(f"Epoch {epoch+1}: loss={epoch_loss:.4f}")
                        
                # 🔹 지표 저장
                accuracy_scores.append(best_acc)
                recall_scores.append(best_recall_macro)
                precision_scores.append(best_precision_macro)
                f1_scores.append(best_f1_macro)
#                ari_scores.append(metrics["ARI"])
                nmi_scores.append(best_nmi)                

            # 🔹 실행 시간 출력
            end_time = time.time()
            print(f"Execution Time for {b11,b12} - node={n}, T={T}: {end_time - start:.5f} sec")
    
            # 🔹 평균 및 표준 편차 계산
            results_summary = {
                "Nodes": n,
                "T": T,
                "epoch" : best_config['epochs'], "dims" : best_config['dims'], "upth_ed" : best_config['upth_ed'],
                "Accuracy_Mean": np.mean(accuracy_scores),
                "Accuracy_Std": np.std(accuracy_scores),
#                "Precision_Mean": np.mean(precision_scores),
#                "Precision_Std": np.std(precision_scores),
#                "Recall_Mean": np.mean(recall_scores),
#                "Recall_Std": np.std(recall_scores),
                "F1-score_Mean": np.mean(f1_scores),
                "F1-score_Std": np.std(f1_scores),
#                "ARI_Mean": np.mean(ari_scores),
#                "ARI_Std": np.std(ari_scores),
                "NMI_Mean": np.mean(nmi_scores),
                "NMI_Std": np.std(nmi_scores),
                'Simulation_Time' : end_time - start
            }
            
            print( results_summary )
        
            # 🔹 DataFrame에 결과 추가
            all_results.append(results_summary)


# ✅ 모든 결과를 DataFrame으로 변환
df_results = pd.DataFrame(all_results)

# ✅ 최종 CSV 파일로 저장 (모든 결과를 한 개의 파일에 저장)           
df_results.to_csv(f"AGE_노스무딩_최최종_0.5_0.01.csv", index=False, encoding="utf-8-sig")
    
print("📁 'simulation_results_summary.csv' 파일 저장 완료!")


✅ 정상적 (Stationary) - 계속 실행 가능

🎯 Best Config: {'bs': 8, 'dims': [32], 'epochs': 50, 'lowth_ed': 0.5, 'lowth_st': 0.3, 'threshold': 0, 'upd': 5, 'upth_ed': 0.03, 'upth_st': 0.1}, DBI=0.6959
Execution Time for (0.5, 0.01) - node=150, T=100: 10033.98698 sec
{'Nodes': 150, 'T': 100, 'epoch': 50, 'dims': [32], 'upth_ed': 0.03, 'Accuracy_Mean': np.float64(0.9068013468013467), 'Accuracy_Std': np.float64(0.08942253849563422), 'F1-score_Mean': np.float64(0.9064806752885186), 'F1-score_Std': np.float64(0.08979522305777232), 'NMI_Mean': np.float64(0.6294332655148337), 'NMI_Std': np.float64(0.2473340275481458), 'Simulation_Time': 10033.986976861954}

🎯 Best Config: {'bs': 8, 'dims': [64], 'epochs': 30, 'lowth_ed': 0.5, 'lowth_st': 0.3, 'threshold': 0, 'upd': 5, 'upth_ed': 0.03, 'upth_st': 0.1}, DBI=0.1718
Execution Time for (0.5, 0.01) - node=150, T=150: 7275.69003 sec
{'Nodes': 150, 'T': 150, 'epoch': 30, 'dims': [64], 'upth_ed': 0.03, 'Accuracy_Mean': np.float64(0.9633670033670035), 'Accuracy_St

In [26]:
import warnings
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from scipy.sparse import diags
from scipy.stats import multivariate_normal
import time 
from sklearn.metrics.pairwise import cosine_similarity
warnings.filterwarnings('ignore')


# ✅ (b11, b12) 조합 리스트
b_combinations = [
#    (0.25, 0.01),
    (0.5, 0.01),
#    (0.25, 0.05),
#    (0.5, 0.1),
]

all_results = []
num_simulations = 100


for b11, b12 in b_combinations:
    for n in [30, 50, 100, 150 ]: # 
        # 파라미터 설정 
        k, phi = 2, 0.99        
        # SBM을 이용한 계수행렬 생성
        sbm_graph = nx.stochastic_block_model([n // k] * k, [[b11 if i == j else b12 for j in range(k)] for i in range(k)], seed=42)
        adj_matrix = nx.to_numpy_array(sbm_graph)
        D_diag = np.sum(adj_matrix, axis=0)  
        D_inv_sqrt = diags(1.0 / np.sqrt(D_diag + 1e-10))
        Phi = phi * D_inv_sqrt @ adj_matrix @ D_inv_sqrt # Φ 생성 

        # 단위근 검정 
        eigenvalues = np.linalg.eigvals(Phi)
        if np.any(np.abs(eigenvalues) >= 1):
            print("⚠ 단위근 존재 (비정상적) - 실행을 중단합니다.")
            sys.exit()  # 실행 중단
        else:
            print("✅ 정상적 (Stationary) - 계속 실행 가능")

        # True label 생성 (test 성능 평가를 위해 쓸 것) 
        True_labels = sum([[i] * (n // k) for i in range(k)], [])
        
        for T in [100, 150, 200, 300, 400, 500, 1000]:

            start = time.time() # 시뮬레이션 시작 타임스탬프

            best_config= tune_hyperparams(True_labels, args, T, n, Phi)
            
            # best_ 성능 저장 초기화
            best_acc, best_nmi, best_adj = 0, 0, 0
            best_f1_macro, best_precision_macro, best_recall_macro = 0, 0, 0
            accuracy_scores = []
            precision_scores = []
            recall_scores = []
            f1_scores = []                #ari_scores = []
            nmi_scores = []
            
#            print('best params로 fitting')

            for i in range(1, num_simulations) :

                np.random.seed(i)                
                Y = np.zeros((T, n))
                Y[0, :] = np.random.normal(size=n)  
                epsilon = multivariate_normal.rvs(mean=np.zeros(n), cov=np.eye(n), size=T)
                for t in range(1, T):
                    Y[t, :] = Phi @ Y[t - 1, :] + epsilon[t, :] 
                              
                corr_matrix = (np.corrcoef(Y.T))
                np.fill_diagonal(corr_matrix, 0)  # self-loop 제거 
                threshold = np.percentile(corr_matrix, best_config["threshold"])
                precision_matrix = (corr_matrix > threshold).astype(int)
                adj = np.array(np.where(precision_matrix > 0)).T  # A 원소가 1인 순서쌍 출력 ([[1 0] [1 1] ...] 인 array 형태) 
                adj = np.array(adj)

                label = True_labels
                adj = sp.csr_matrix((np.ones(len(adj)), (adj[:,0], adj[:,1])), shape=(len(label), len(label)))
                
                features = torch.FloatTensor(MinMaxScaler().fit_transform(Y.T))
                    
                Cluster = KMeans(n_clusters=k, random_state=i)
                db, nmi, adjscore, acc, f1_macro, precision_macro, recall_macro = clustering(Cluster, features, True_labels) 
                best_cl = db
                adj_1st = (adj + sp.eye(n)).toarray()
                adj_label = torch.FloatTensor(adj_1st)
                adj_label = adj_label.reshape([-1,]) #1차원 벡터로 펴줌

                # 모델 정의
                dims = [features.shape[1]] + best_config["dims"]
                model = LinTrans(args.linlayers, dims)
                optimizer = optim.Adam(model.parameters(), lr=0.01) # Adam으로 최적화
                if args.cuda:
                    model.cuda()
                    inx = features.cuda() # 스무딩된 행렬 X틸다를 행 단위로 array 
                    adj_label = adj_label.cuda()

                pos_num = len(adj.indices) # 양수 샘플 개수 정의 
                n_nodes = adj.shape[0]
                neg_num = n_nodes*n_nodes-pos_num # 음수 샘플 개수 정의 
            
                up_eta = (best_config["upth_ed"] - best_config["upth_st"]) / (best_config["epochs"]/best_config["upd"]) # 몇 번의 업데이트에서 변화(eta)를 적용할 것인지 계산
                low_eta = (best_config["lowth_ed"] - best_config["lowth_st"]) / (best_config["epochs"]/best_config["upd"]) # 몇 번의 업데이트에서 변화(eta)를 적용할 것인지 계산
            
                #처음엔 인접행렬에서 pos_num , neg_num 정의 
                pos_inds, neg_inds = update_similarity(normalize(features.numpy()), best_config["upth_st"], best_config["lowth_st"], pos_num, neg_num) 
                upth, lowth = update_threshold(best_config["upth_st"], best_config["lowth_st"], up_eta, low_eta) # 임계값 업데이트 
            
                bs = min(best_config["bs"], len(pos_inds)) # 배치 사이즈 : bs
                length = len(pos_inds)
                
                pos_inds_cuda = torch.LongTensor(pos_inds).cuda()
                for epoch in (range(best_config["epochs"])):
#                    print(epoch+1)
                    
                    st, ed = 0, bs # bs가 len(pos_inds) ed 
                    batch_num = 0
                    model.train()
                    length = len(pos_inds)
                    epoch_loss = 0
                    
                    while ( ed <= length ): # 배치 사이즈가 pos 길이보다 작거나 같은 동안 
                        sampled_neg = torch.LongTensor(np.random.choice(neg_inds, size=ed-st)).cuda() # neg_inds에서 ed-st 크기 랜덤 샘플하여 sampled_neg 
                        sampled_inds = torch.cat((pos_inds_cuda[st:ed], sampled_neg), 0) # 현재 배치에서 생성된 양성 샘플과 음성 샘플 합체 
                        optimizer.zero_grad() # 그래디언트 초기화 
                        xind = sampled_inds // n_nodes
                        yind = sampled_inds % n_nodes
                        x = torch.index_select(inx, 0, xind) # 샘플링된 노드 쌍에서 첫번째 노드의 인덱스 추출 
                        y = torch.index_select(inx, 0, yind) # 샘플링된 노드 쌍에서 두번째 노드의 인덱스 추출  - 각각의 노드 feature가 됨 
                        zx = model(x) # 모델을 거친 후의 노드 임베딩 
                        zy = model(y) # 모델을 거친 후의 노드 임베딩 
                        batch_label = torch.cat((torch.ones(ed-st), torch.zeros(ed-st)), dim=0).cuda() # 라벨 생성 
                        batch_pred = model.dcs(zx, zy) #  두 노드 간 유사도 계산 (link_pred) 
                        loss = loss_function(adj_preds=batch_pred, adj_labels=batch_label, n_nodes=ed-st) # 예측한 링크 확률과 실제 정답 간 차이, 최소화하도록 모델 학습 
                        
                        loss.backward() # 손실 기반 그래디언트 계산 
                        cur_loss = loss.item() # 현재 배치의 손실값 저장 
                        optimizer.step() # 모델 가중치 업데이트 
                        epoch_loss += loss.item()  # 손실 누적
                        
                        st = ed # 다음 배치 시작 위치 업데이트 
                        batch_num += 1
                        if ed < length and ed + bs >= length: 
                            ed += length - ed
                        else:
                            ed += bs
                
                    if (epoch + 1) % best_config["upd"] == 0: # 업데이트 횟수 T의 배수번째인 에폭마다 전체 데이터
                        model.eval() # 평가 모드로 전환 
                        mu = model(inx) #전체 feature_matrix의 행을 변환 
                        hidden_emb = mu.cpu().data.numpy()
                        upth, lowth = update_threshold(upth, lowth, up_eta, low_eta) # threshold 업데이트하여 threshold 새로 정의  
                        pos_inds, neg_inds = update_similarity(hidden_emb, upth, lowth, pos_num, neg_num) # 새로 정의된 threshold로 pos neg 샘플 인덱스 (imbalanced)
                        bs = min(best_config["bs"], len(pos_inds)) #bs 새로 정의 - pos_num 새로 정의돼서 
                        pos_inds_cuda = torch.LongTensor(pos_inds).cuda()
                        db, nmi, adjscore, acc, f1_macro, precision_macro, recall_macro = clustering(Cluster, hidden_emb, True_labels) # 지금까지 학습된 모델로 X틸다 확장시킨 행렬로 클러스터링
                        if db >= best_cl:
                            best_cl = db
                            best_acc, best_nmi, best_adj = acc, nmi, adjscore
                            best_f1_macro, best_precision_macro, best_recall_macro = f1_macro, precision_macro, recall_macro
#                    tqdm.write(f"Epoch {epoch+1}: loss={epoch_loss:.4f}")
                        
                # 🔹 지표 저장
                accuracy_scores.append(best_acc)
                recall_scores.append(best_recall_macro)
                precision_scores.append(best_precision_macro)
                f1_scores.append(best_f1_macro)
#                ari_scores.append(metrics["ARI"])
                nmi_scores.append(best_nmi)                

            # 🔹 실행 시간 출력
            end_time = time.time()
            print(f"Execution Time for {b11,b12} - node={n}, T={T}: {end_time - start:.5f} sec")
    
            # 🔹 평균 및 표준 편차 계산
            results_summary = {
                "Nodes": n,
                "T": T,
                "epoch" : best_config['epochs'], "dims" : best_config['dims'], "upth_ed" : best_config['upth_ed'],
                "Accuracy_Mean": np.mean(accuracy_scores),
                "Accuracy_Std": np.std(accuracy_scores),
#                "Precision_Mean": np.mean(precision_scores),
#                "Precision_Std": np.std(precision_scores),
#                "Recall_Mean": np.mean(recall_scores),
#                "Recall_Std": np.std(recall_scores),
                "F1-score_Mean": np.mean(f1_scores),
                "F1-score_Std": np.std(f1_scores),
#                "ARI_Mean": np.mean(ari_scores),
#                "ARI_Std": np.std(ari_scores),
                "NMI_Mean": np.mean(nmi_scores),
                "NMI_Std": np.std(nmi_scores),
                'Simulation_Time' : end_time - start
            }
            
            print( results_summary )
        
            # 🔹 DataFrame에 결과 추가
            all_results.append(results_summary)


# ✅ 모든 결과를 DataFrame으로 변환
df_results = pd.DataFrame(all_results)

# ✅ 최종 CSV 파일로 저장 (모든 결과를 한 개의 파일에 저장)           
df_results.to_csv(f"AGE_노스무딩_최최종_0.5_0.01.csv", index=False, encoding="utf-8-sig")
    
print("📁 'simulation_results_summary.csv' 파일 저장 완료!")


✅ 정상적 (Stationary) - 계속 실행 가능

🎯 Best Config: {'bs': 8, 'dims': [64], 'epochs': 30, 'lowth_ed': 0.5, 'lowth_st': 0.3, 'threshold': 0, 'upd': 5, 'upth_ed': 0.05, 'upth_st': 0.1}, DBI=0.0980
Execution Time for (0.5, 0.01) - node=30, T=100: 395.22425 sec
{'Nodes': 30, 'T': 100, 'epoch': 30, 'dims': [64], 'upth_ed': 0.05, 'Accuracy_Mean': np.float64(0.9804713804713804), 'Accuracy_Std': np.float64(0.04738032973259099), 'F1-score_Mean': np.float64(0.9802852329357361), 'F1-score_Std': np.float64(0.04803680643864912), 'NMI_Mean': np.float64(0.920429865695429), 'NMI_Std': np.float64(0.14970047817037754), 'Simulation_Time': 395.22424721717834}

🎯 Best Config: {'bs': 8, 'dims': [64], 'epochs': 45, 'lowth_ed': 0.5, 'lowth_st': 0.3, 'threshold': 0, 'upd': 5, 'upth_ed': 0.05, 'upth_st': 0.1}, DBI=0.0377
Execution Time for (0.5, 0.01) - node=30, T=150: 365.57048 sec
{'Nodes': 30, 'T': 150, 'epoch': 45, 'dims': [64], 'upth_ed': 0.05, 'Accuracy_Mean': np.float64(0.992929292929293), 'Accuracy_Std': np.f

KeyboardInterrupt: 

In [30]:

# ✅ 모든 결과를 DataFrame으로 변환

df_results = pd.DataFrame(all_results)

miss= df_results

In [11]:

# ✅ 모든 결과를 DataFrame으로 변환
df_results = pd.DataFrame(all_results)

# ✅ 최종 CSV 파일로 저장 (모든 결과를 한 개의 파일에 저장)           
df_results.to_csv(f"AGE_노스무딩_최최종_0.5_0.01_100.csv", index=False, encoding="utf-8-sig")
    
print("📁 'simulation_results_summary.csv' 파일 저장 완료!")

📁 'simulation_results_summary.csv' 파일 저장 완료!


In [14]:
df_results_1 = df_results.copy()

In [15]:
df_results_1

,b11,b12,Nodes,T,epoch,dims,threshold,Accuracy_Mean,Accuracy_Std,Precision_Mean,Precision_Std,Recall_Mean,Recall_Std,F1-score_Mean,F1-score_Std,NMI_Mean,NMI_Std,Simulation_Time
0,0.5,0.01,30,100,30,[64],0,0.978451,0.053036,0.980241,0.050820,0.978451,0.053036,0.978344,0.053259,0.913008,0.161359,335.758792
1,0.5,0.01,30,150,50,[64],0,0.992256,0.022135,0.993210,0.019195,0.992256,0.022135,0.992213,0.022278,0.966276,0.093662,450.536507
2,0.5,0.01,30,200,30,[32],0,0.992929,0.019690,0.993699,0.017301,0.992929,0.019690,0.992899,0.019796,0.967658,0.086632,334.395239
3,0.5,0.01,30,300,50,[64],0,0.997643,0.011847,0.997902,0.010503,0.997643,0.011847,0.997634,0.011897,0.989361,0.052772,457.486398
4,0.5,0.01,30,400,45,[64],0,0.998316,0.011961,0.998564,0.010151,0.998316,0.011961,0.998303,0.012061,0.993206,0.047738,663.515412
5,0.5,0.01,30,500,50,[64],0,0.998990,0.009999,0.999158,0.008333,0.998990,0.009999,0.998980,0.010100,0.996151,0.038100,773.004813
6,0.5,0.01,30,1000,50,[32],0,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,851.036500
7,0.5,0.01,50,100,45,[32],0,0.968081,0.050286,0.970278,0.047532,0.968081,0.050286,0.967923,0.050720,0.859713,0.185316,1093.721335
8,0.5,0.01,50,150,45,[32],0,0.993131,0.018676,0.993678,0.016699,0.993131,0.018676,0.993107,0.018787,0.965400,0.085006,961.880349
9,0.5,0.01,50,200,50,[32],0,0.997980,0.006664,0.998072,0.006326,0.997980,0.006664,0.997979,0.006669,0.988099,0.038430,1854.977008


In [126]:
# hyperparam_tuning_fit.py

import numpy as np
import scipy.sparse as sp
import torch
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.model_selection import ParameterGrid

# 필요한 사용자 정의 함수들은 외부에서 import 하거나 별도 구현 필요
# from your_module import LinTrans, preprocess_graph, clustering, update_similarity, update_threshold

def tune_hyperparams(true_labels, args, T, n, Phi):
    best_score = -np.inf
    best_config = None
    best_model_state = None

    param_grid = {
        "dims": [[n]],
        "threshold": [80, 85, 90],
        "epochs": [20, 25, 30, 35, 40, 45, 50, 55, 60]}

    for config in ParameterGrid(param_grid):
        print(f"⏳ Trying config: {config}")
        np.random.seed(117)  # 시드 고정
        train_size = int(T * 0.8)  # 시뮬레이션 시작 전 train 길이 설정 
        Y = np.zeros((T, n))
        Y[0, :] = np.random.normal(size=n)  
        epsilon = multivariate_normal.rvs(mean=np.zeros(n), cov=np.eye(n), size=T)
        for t in range(1, T):
            Y[t, :] = Phi @ Y[t - 1, :] + epsilon[t, :] # Y[t - 1, :]가 t-1행을 열벡터로 표현하여 계수행렬 Phi와 곱해짐 , 행이 위로 갈수록 과거 
        
        # 전체 데이터로 인접행렬 추정 + 라벨 
        corr_matrix = (np.corrcoef(Y.T)) 
        np.fill_diagonal(corr_matrix, 0)  # self-loop 제거 
        threshold = np.percentile(corr_matrix, config["threshold"]) 
        precision_matrix = (corr_matrix > threshold).astype(int)

#        precision_matrix = (np.corrcoef(Y.T))

        
        adj = np.array(np.where(precision_matrix > 0)).T  # A 원소가 1인 순서쌍 출력 ([[1 0] [1 1] ...] 인 array 형태) 
        adj = sp.csr_matrix((np.ones(len(adj)), (adj[:,0], adj[:,1])), shape=(len(true_labels), len(true_labels))) # 추정된 A 
        
        # Train/Test 분리 (8:2) 후 사이즈 맞추기 
        train_data, test_data = Y[:train_size], Y[train_size:] 
        train_len, feature_dim = train_data.shape

        features = torch.FloatTensor(MinMaxScaler().fit_transform(train_data.T))
        adj = adj - sp.dia_matrix((adj.diagonal()[np.newaxis, :], [0]), shape=adj.shape)
        adj.eliminate_zeros()
        # 스무딩 후
        adj_1st = (adj + sp.eye(n)).toarray()
        adj_label = torch.FloatTensor(adj_1st).reshape([-1,])

        pos_num = len(adj.indices)
        neg_num = n * n - pos_num

        up_eta = (args.upth_ed - args.upth_st) / (config["epochs"] / args.upd)
        low_eta = (args.lowth_ed - args.lowth_st) / (config["epochs"] / args.upd)

#        pos_inds, neg_inds = update_similarity(features.numpy(), args.upth_st, args.lowth_st, pos_num, neg_num) # 
        pos_inds, neg_inds = update_similarity_1(precision_matrix, args.upth_st, args.lowth_st, pos_num, neg_num) 
        upth, lowth = update_threshold(args.upth_st, args.lowth_st, up_eta, low_eta) 

        bs = min(args.bs, len(pos_inds))
        length = len(pos_inds)

        dims = [features.shape[1]] + config["dims"]
        model = LinTrans(args.linlayers, dims)
        optimizer = optim.Adam(model.parameters(), lr=0.01)

        if args.cuda:
            model.cuda()
            inx = features.cuda()
            adj_label = adj_label.cuda()
        else:
            inx = features

        pos_inds_cuda = torch.LongTensor(pos_inds)
        if args.cuda:
            pos_inds_cuda = pos_inds_cuda.cuda()

        print('튜닝을 위한 Start Training...')
        
        for epoch in range(config['epochs']):
            print(f'{epoch+1}th , pos_length = {length}')
            st, ed = 0, bs # bs가 len(pos_inds) ed 
            batch_num = 0
            model.train()
            length = len(pos_inds)
            while ed <= length:
                print(f'{epoch} 번째 에폭 , ed :{ed} , length: {ed - st}')
                sampled_neg = torch.LongTensor(np.random.choice(neg_inds, size=ed-st))
                if args.cuda:
                    sampled_neg = sampled_neg.cuda()
                sampled_inds = torch.cat((pos_inds_cuda[st:ed], sampled_neg), 0)

                optimizer.zero_grad()
                xind = sampled_inds // n
                yind = sampled_inds % n
                x = torch.index_select(inx, 0, xind)
                y = torch.index_select(inx, 0, yind)
                zx = model(x)
                zy = model(y)
                batch_label = torch.cat((torch.ones(ed-st), torch.zeros(ed-st)), dim=0).cuda() # 라벨 생성 
                if args.cuda:
                    batch_label = batch_label.cuda()
                batch_pred = model.dcs(zx, zy)
                loss = loss_function(adj_preds=batch_pred, adj_labels=batch_label, n_nodes=ed-st)
                loss.backward()
                optimizer.step()

                st = ed
                if ed < length and ed + bs >= length: 
                    ed += length - ed
                else:
                    ed += bs

                batch_num+=1

            if (epoch + 1) % args.upd == 0:
                model.eval()
                mu = model(inx)
                hidden_emb = mu.detach().cpu().numpy()
                upth, lowth = update_threshold(upth, lowth, up_eta, low_eta)
                precision_matrix = (np.corrcoef(hidden_emb))             
                pos_inds, neg_inds = update_similarity_1(precision_matrix, args.upth_st, args.lowth_st, pos_num, neg_num) 
                bs = min(args.bs, len(pos_inds))
                pos_inds_cuda = torch.LongTensor(pos_inds)
                if args.cuda:
                    pos_inds_cuda = pos_inds_cuda.cuda()

                db, nmi, adjscore, acc, f1_macro, precision_macro, recall_macro = clustering(KMeans(n_clusters=2), hidden_emb, True_labels)

                if db > best_score:
                    best_score = db
                    best_config = config
                    best_model_state = model.state_dict()
            print(f'배치 회수: {batch_num}') 
            
                    

    print(f"\n🎯 Best Config: {best_config}, DBI={-best_score:.4f}")
    return best_config

In [127]:
import warnings
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from scipy.sparse import diags
from scipy.stats import multivariate_normal
import time 
from sklearn.metrics.pairwise import cosine_similarity
warnings.filterwarnings('ignore')


# ✅ (b11, b12) 조합 리스트
b_combinations = [
    (0.25, 0.01),
    (0.5, 0.01),
    (0.25, 0.05),
    (0.5, 0.1),
]

all_results = []
num_simulations = 100


for b11, b12 in b_combinations:


    for n in [50 ]: #30, 50, 100, 150
        # 파라미터 설정 
        k, phi = 2, 0.99
        
        # SBM을 이용한 계수행렬 생성
        sbm_graph = nx.stochastic_block_model([n // k] * k, [[b11 if i == j else b12 for j in range(k)] for i in range(k)], seed=42)
        adj_matrix = nx.to_numpy_array(sbm_graph)
        D_diag = np.sum(adj_matrix, axis=0)  
        D_inv_sqrt = diags(1.0 / np.sqrt(D_diag + 1e-10))
        Phi = phi * D_inv_sqrt @ adj_matrix @ D_inv_sqrt # Φ 생성 

        # 단위근 검정 
        eigenvalues = np.linalg.eigvals(Phi)
        if np.any(np.abs(eigenvalues) >= 1):
            print("⚠ 단위근 존재 (비정상적) - 실행을 중단합니다.")
            sys.exit()  # 실행 중단
        else:
            print("✅ 정상적 (Stationary) - 계속 실행 가능")

        # True label 생성 (test 성능 평가를 위해 쓸 것) 
        True_labels = sum([[i] * (n // k) for i in range(k)], [])
        
        for T in [100]: # 100, 150, 200, 300, 400, 500, 1000
            best_config= tune_hyperparams(True_labels, args, T, n, Phi)
            
            # best_ 성능 저장 초기화
            best_acc, best_nmi, best_adj = 0, 0, 0
            best_f1_macro, best_precision_macro, best_recall_macro = 0, 0, 0
            accuracy_scores = []
            precision_scores = []
            recall_scores = []
            f1_scores = []                
            #ari_scores = []
            nmi_scores = []
            
            print('best params로 fitting')
            start = time.time() # 시뮬레이션 시작 타임스탬프

            for i in range(1, num_simulations) :

                np.random.seed(i)                
                Y = np.zeros((T, n))
                Y[0, :] = np.random.normal(size=n)  
                epsilon = multivariate_normal.rvs(mean=np.zeros(n), cov=np.eye(n), size=T)
                for t in range(1, T):
                    Y[t, :] = Phi @ Y[t - 1, :] + epsilon[t, :] 
                              
                precision_matrix = np.abs(np.corrcoef(Y.T))
                #np.fill_diagonal(corr_matrix, 0)  # self-loop 제거 
                #threshold = np.percentile(corr_matrix, best_config["threshold"])
                #precision_matrix = (corr_matrix > threshold).astype(int)

#                precision_matrix = np.abs(np.corrcoef(Y.T))

                
                adj = np.array(np.where(precision_matrix > 0)).T  # A 원소가 1인 순서쌍 출력 ([[1 0] [1 1] ...] 인 array 형태) 
                adj = np.array(adj)

                label = True_labels
                adj = sp.csr_matrix((np.ones(len(adj)), (adj[:,0], adj[:,1])), shape=(len(label), len(label)))
                
                features = torch.FloatTensor(MinMaxScaler().fit_transform(Y.T))
                    
                Cluster = KMeans(n_clusters=k, random_state=i)
                db, nmi, adjscore, acc, f1_macro, precision_macro, recall_macro = clustering(Cluster, features, True_labels) 
                best_cl = db
                adj_1st = (adj + sp.eye(n)).toarray()
                adj_label = torch.FloatTensor(adj_1st)
                adj_label = adj_label.reshape([-1,]) #1차원 벡터로 펴줌

                # 모델 정의
                dims = [features.shape[1]] + best_config["dims"]
                model = LinTrans(args.linlayers, dims)
                optimizer = optim.Adam(model.parameters(), lr=0.01) # Adam으로 최적화
                if args.cuda:
                    model.cuda()
                    inx = features.cuda() # 스무딩된 행렬 X틸다를 행 단위로 array 
                    adj_label = adj_label.cuda()

                pos_num = len(adj.indices) # 양수 샘플 개수 정의 
                n_nodes = adj.shape[0]
                neg_num = n_nodes*n_nodes-pos_num # 음수 샘플 개수 정의 
            
                up_eta = (args.upth_ed - args.upth_st) / (best_config["epochs"]/args.upd) # 몇 번의 업데이트에서 변화(eta)를 적용할 것인지 계산
                low_eta = (args.lowth_ed - args.lowth_st) / (best_config["epochs"]/args.upd) # 몇 번의 업데이트에서 변화(eta)를 적용할 것인지 계산
            
                #처음엔 인접행렬에서 pos_num , neg_num 정의 
                pos_inds, neg_inds = update_similarity_1(precision_matrix, args.upth_st, args.lowth_st, pos_num, neg_num) 
                upth, lowth = update_threshold(args.upth_st, args.lowth_st, up_eta, low_eta) # 임계값 업데이트 
            
                bs = min(args.bs, len(pos_inds)) # 배치 사이즈 : bs
                length = len(pos_inds)
                
                pos_inds_cuda = torch.LongTensor(pos_inds).cuda()
                for epoch in tqdm(range(best_config["epochs"])):
                    
                    st, ed = 0, bs # bs가 len(pos_inds) ed 
                    batch_num = 0
                    model.train()
                    length = len(pos_inds)
                    epoch_loss = 0
                    
                    while ( ed <= length ): # 배치 사이즈가 pos 길이보다 작거나 같은 동안 
                        sampled_neg = torch.LongTensor(np.random.choice(neg_inds, size=ed-st)).cuda() # neg_inds에서 ed-st 크기 랜덤 샘플하여 sampled_neg 
                        sampled_inds = torch.cat((pos_inds_cuda[st:ed], sampled_neg), 0) # 현재 배치에서 생성된 양성 샘플과 음성 샘플 합체 
                        optimizer.zero_grad() # 그래디언트 초기화 
                        xind = sampled_inds // n_nodes
                        yind = sampled_inds % n_nodes
                        x = torch.index_select(inx, 0, xind) # 샘플링된 노드 쌍에서 첫번째 노드의 인덱스 추출 
                        y = torch.index_select(inx, 0, yind) # 샘플링된 노드 쌍에서 두번째 노드의 인덱스 추출  - 각각의 노드 feature가 됨 
                        zx = model(x) # 모델을 거친 후의 노드 임베딩 
                        zy = model(y) # 모델을 거친 후의 노드 임베딩 
                        batch_label = torch.cat((torch.ones(ed-st), torch.zeros(ed-st)), dim=0).cuda() # 라벨 생성 
                        batch_pred = model.dcs(zx, zy) #  두 노드 간 유사도 계산 (link_pred) 
                        loss = loss_function(adj_preds=batch_pred, adj_labels=batch_label, n_nodes=ed-st) # 예측한 링크 확률과 실제 정답 간 차이, 최소화하도록 모델 학습 
                        
                        loss.backward() # 손실 기반 그래디언트 계산 
                        cur_loss = loss.item() # 현재 배치의 손실값 저장 
                        optimizer.step() # 모델 가중치 업데이트 
                        epoch_loss += loss.item()  # 손실 누적
                        
                        st = ed # 다음 배치 시작 위치 업데이트 
                        batch_num += 1
                        if ed < length and ed + bs >= length: 
                            ed += length - ed
                        else:
                            ed += bs
                
                    if (epoch + 1) % args.upd == 0: # 업데이트 횟수 T의 배수번째인 에폭마다 전체 데이터
                        model.eval() # 평가 모드로 전환 
                        mu = model(inx) #전체 feature_matrix의 행을 변환 
                        hidden_emb = mu.cpu().data.numpy()
                        upth, lowth = update_threshold(upth, lowth, up_eta, low_eta) # threshold 업데이트하여 threshold 새로 정의  

                        precision_matrix = (np.corrcoef(hidden_emb))

                        pos_inds, neg_inds = update_similarity_1(precision_matrix, upth, lowth, pos_num, neg_num) # 새로 정의된 threshold로 pos neg 샘플 인덱스 (imbalanced)
                        bs = min(args.bs, len(pos_inds)) #bs 새로 정의 - pos_num 새로 정의돼서 
                        pos_inds_cuda = torch.LongTensor(pos_inds).cuda()
                        db, nmi, adjscore, acc, f1_macro, precision_macro, recall_macro = clustering(Cluster, hidden_emb, True_labels) # 지금까지 학습된 모델로 X틸다 확장시킨 행렬로 클러스터링
                        if db >= best_cl:
                            best_cl = db
                            best_acc, best_nmi, best_adj = acc, nmi, adjscore
                            best_f1_macro, best_precision_macro, best_recall_macro = f1_macro, precision_macro, recall_macro
#                    tqdm.write(f"Epoch {epoch+1}: loss={epoch_loss:.4f}")
                        
                # 🔹 지표 저장
                accuracy_scores.append(best_acc)
                recall_scores.append(best_recall_macro)
                precision_scores.append(best_precision_macro)
                f1_scores.append(best_f1_macro)
#                ari_scores.append(metrics["ARI"])
                nmi_scores.append(best_nmi)                

            # 🔹 실행 시간 출력
            end_time = time.time()
            print(f"Execution Time for {b11,b12} - node={n}, T={T}: {end_time - start:.5f} sec")
    
            # 🔹 평균 및 표준 편차 계산
            results_summary = {
                'b11':b11,
                'b12':b12,
                "Nodes": n,
                "T": T,
                "epoch" : best_config['epochs'], "dims" : best_config['dims'], "threshold" : best_config['threshold'],
                "Accuracy_Mean": np.mean(accuracy_scores),
                "Accuracy_Std": np.std(accuracy_scores),
                "Precision_Mean": np.mean(precision_scores),
                "Precision_Std": np.std(precision_scores),
                "Recall_Mean": np.mean(recall_scores),
                "Recall_Std": np.std(recall_scores),
                "F1-score_Mean": np.mean(f1_scores),
                "F1-score_Std": np.std(f1_scores),
#                "ARI_Mean": np.mean(ari_scores),
#                "ARI_Std": np.std(ari_scores),
                "NMI_Mean": np.mean(nmi_scores),
                "NMI_Std": np.std(nmi_scores),
                'Simulation_Time' : end_time - start
            }
    

            print( results_summary )
        
            # 🔹 DataFrame에 결과 추가
            all_results.append(results_summary)


# ✅ 모든 결과를 DataFrame으로 변환
df_results = pd.DataFrame(all_results)

# ✅ 최종 CSV 파일로 저장 (모든 결과를 한 개의 파일에 저장)
df_results.to_csv(f"피드백_상관계수.csv", index=False, encoding="utf-8-sig")
    
print("📁 'simulation_results_summary.csv' 파일 저장 완료!")


✅ 정상적 (Stationary) - 계속 실행 가능
⏳ Trying config: {'dims': [50], 'epochs': 20, 'threshold': 80}


RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [126]:
df_results = pd.DataFrame(all_results)
df_results

,b11,b12,Nodes,T,epoch,dims,threshold,Accuracy_Mean,Accuracy_Std,Precision_Mean,Precision_Std,Recall_Mean,Recall_Std,F1-score_Mean,F1-score_Std,NMI_Mean,NMI_Std,Simulation_Time
0,0.5,0.01,30,100,35,[30],80,0.975510,0.073702,0.977141,0.072769,0.975510,0.073702,0.974857,0.077550,0.915587,0.176047,29.507214
1,0.5,0.01,30,150,30,[30],85,0.992517,0.019361,0.993285,0.017238,0.992517,0.019361,0.992490,0.019438,0.965266,0.087666,22.504620
2,0.5,0.01,30,200,35,[30],85,0.995238,0.017817,0.995823,0.015322,0.995238,0.017817,0.995210,0.017951,0.978985,0.074549,24.870594
3,0.5,0.01,30,300,35,[30],80,0.997959,0.014139,0.998299,0.011783,0.997959,0.014139,0.997939,0.014282,0.992224,0.053874,28.300365
4,0.5,0.01,30,400,35,[30],85,0.998639,0.009426,0.998800,0.008317,0.998639,0.009426,0.998633,0.009468,0.994050,0.041224,28.050701
5,0.5,0.01,30,500,35,[30],90,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,25.266075
6,0.5,0.01,30,1000,35,[30],80,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,28.766720
7,0.5,0.01,50,100,35,[50],80,0.965714,0.050467,0.969638,0.044089,0.965714,0.050467,0.965405,0.051104,0.857000,0.188941,57.101954
8,0.5,0.01,50,150,35,[50],85,0.992245,0.020924,0.992993,0.018441,0.992245,0.020924,0.992210,0.021057,0.963130,0.092481,54.917492
9,0.5,0.01,50,200,25,[50],85,0.999184,0.003957,0.999215,0.003805,0.999184,0.003957,0.999183,0.003959,0.995029,0.024099,39.186595


In [127]:
# ✅ 최종 CSV 파일로 저장 (모든 결과를 한 개의 파일에 저장)
df_results.to_csv(f"AGE_노스무딩_튜닝 후 전체 학습_0.5&0.01_150까지.csv", index=False, encoding="utf-8-sig")
    
print("📁 'simulation_results_summary.csv' 파일 저장 완료!")

📁 'simulation_results_summary.csv' 파일 저장 완료!


In [120]:
# ✅ 모든 결과를 DataFrame으로 변환
df_results = pd.DataFrame(all_results)
df_results.to_csv(f"AGE_노스무딩_튜닝 후 전체 학습_0.5&0.1_150까지만 .csv", index=False, encoding="utf-8-sig")


In [59]:
# ✅ 모든 결과를 DataFrame으로 변환
df_results = pd.DataFrame(all_results)
df_results

,b11,b12,Nodes,T,epoch,dims,threshold,Accuracy_Mean,Accuracy_Std,Precision_Mean,Precision_Std,Recall_Mean,Recall_Std,F1-score_Mean,F1-score_Std,NMI_Mean,NMI_Std,Simulation_Time
0,0.25,0.05,30,100,35,[500],90,0.559259,0.073330,0.662341,0.109744,0.559259,0.073330,0.492844,0.109850,0.051091,0.063919,5.101799
1,0.25,0.05,30,150,35,[500],90,0.585185,0.225044,0.627192,0.241425,0.585185,0.225044,0.560463,0.225129,0.128110,0.112344,6.766897
2,0.25,0.05,30,200,35,[500],90,0.618519,0.251470,0.656935,0.253899,0.618519,0.251470,0.595600,0.259323,0.172595,0.174658,5.389857
3,0.25,0.05,30,300,35,[500],90,0.785185,0.094426,0.816716,0.050187,0.785185,0.094426,0.772231,0.117756,0.311171,0.122110,3.849409
4,0.25,0.05,30,400,35,[500],90,0.711111,0.138778,0.734100,0.138277,0.711111,0.138778,0.680433,0.176603,0.229125,0.174580,7.584460


In [54]:
# 차원 10배, 0.1, 0.01
# ✅ 모든 결과를 DataFrame으로 변환
df_results = pd.DataFrame(all_results)
df_results

,b11,b12,Nodes,T,epoch,dims,threshold,Accuracy_Mean,Accuracy_Std,Precision_Mean,Precision_Std,Recall_Mean,Recall_Std,F1-score_Mean,F1-score_Std,NMI_Mean,NMI_Std,Simulation_Time
0,0.5,0.1,30,150,30,[1500],85,0.619728,0.130048,0.693759,0.128662,0.619728,0.130048,0.565521,0.155571,0.151303,0.251785,56.699725
1,0.5,0.1,30,200,30,[2000],90,0.610204,0.117457,0.679630,0.129818,0.610204,0.117457,0.552298,0.144544,0.130481,0.184863,38.702027
2,0.5,0.1,30,300,35,[3000],80,0.622449,0.136425,0.687303,0.133461,0.622449,0.136425,0.567502,0.166256,0.148261,0.238835,47.324016
3,0.5,0.1,30,400,35,[4000],90,0.653741,0.153835,0.722838,0.142483,0.653741,0.153835,0.600313,0.188511,0.209258,0.282174,58.123094
4,0.5,0.1,30,500,35,[5000],90,0.672789,0.168584,0.735118,0.155168,0.672789,0.168584,0.623113,0.205559,0.245618,0.295749,61.939206
5,0.5,0.1,30,1000,30,[10000],90,0.773469,0.206798,0.816339,0.176475,0.773469,0.206798,0.734174,0.251105,0.464364,0.393768,58.338568
6,0.5,0.1,50,150,30,[1500],85,0.566122,0.067878,0.610521,0.082406,0.566122,0.067878,0.510757,0.090941,0.041719,0.063170,96.349184
7,0.5,0.1,50,200,20,[2000],85,0.580408,0.083053,0.646379,0.098307,0.580408,0.083053,0.515318,0.115130,0.067593,0.091964,55.412060


In [92]:
# 임계치 상위 5%, 1% , bs = 16
df_results = pd.DataFrame(all_results)
df_results

,b11,b12,Nodes,T,epoch,dims,threshold,Accuracy_Mean,Accuracy_Std,Precision_Mean,Precision_Std,Recall_Mean,Recall_Std,F1-score_Mean,F1-score_Std,NMI_Mean,NMI_Std,Simulation_Time
0,0.5,0.1,30,100,25,[400],90,0.672109,0.136584,0.723351,0.126380,0.672109,0.136584,0.638066,0.164622,0.197092,0.234883,17.561248
1,0.5,0.1,30,150,30,[1500],90,0.642177,0.147200,0.725342,0.137403,0.642177,0.147200,0.583583,0.183056,0.193385,0.256717,14.464171
2,0.5,0.1,30,200,25,[1000],90,0.583673,0.102159,0.668948,0.121273,0.583673,0.102159,0.511563,0.131465,0.097769,0.167027,11.228631
3,0.5,0.1,30,300,30,[600],90,0.640816,0.144006,0.719600,0.140097,0.640816,0.144006,0.582382,0.178288,0.190647,0.247466,13.954981
4,0.5,0.1,30,400,20,[800],90,0.625170,0.144676,0.726559,0.133112,0.625170,0.144676,0.555674,0.179883,0.181949,0.268478,9.516339
5,0.5,0.1,30,500,35,[1000],90,0.649660,0.167294,0.714979,0.157199,0.649660,0.167294,0.594675,0.201886,0.215748,0.303037,16.588267
6,0.5,0.1,30,1000,25,[2000],90,0.668707,0.207283,0.746045,0.187004,0.668707,0.207283,0.610287,0.244668,0.278634,0.334560,13.201299
7,0.5,0.1,50,100,35,[1000],90,0.568571,0.065776,0.613265,0.084420,0.568571,0.065776,0.514220,0.089624,0.044770,0.063723,29.762933
8,0.5,0.1,50,150,35,[450],90,0.585714,0.099632,0.647841,0.110885,0.585714,0.099632,0.528763,0.133988,0.073402,0.126362,30.253896
9,0.5,0.1,50,200,30,[1000],90,0.567347,0.082283,0.631090,0.098105,0.567347,0.082283,0.495612,0.117152,0.052943,0.095675,26.664386


In [60]:
# 임곛
df_results = pd.DataFrame(all_results)
df_results

,b11,b12,Nodes,T,epoch,dims,threshold,Accuracy_Mean,Accuracy_Std,Precision_Mean,Precision_Std,Recall_Mean,Recall_Std,F1-score_Mean,F1-score_Std,NMI_Mean,NMI_Std,Simulation_Time
0,0.5,0.1,30,100,35,[1000],90,0.674830,0.150965,0.704814,0.146025,0.674830,0.150965,0.647620,0.173046,0.202185,0.245948,66.755252
1,0.5,0.1,30,150,25,[1500],80,0.633333,0.137354,0.714235,0.131938,0.633333,0.137354,0.574773,0.171820,0.171973,0.241335,21.375946
2,0.5,0.1,30,200,30,[2000],90,0.612925,0.130204,0.671893,0.133835,0.612925,0.130204,0.560864,0.154697,0.133364,0.220502,62.121139
3,0.5,0.1,30,300,20,[3000],85,0.668027,0.161338,0.743259,0.139383,0.668027,0.161338,0.615624,0.198017,0.229690,0.285933,41.461419
4,0.5,0.1,30,400,35,[4000],90,0.644218,0.162239,0.714144,0.147270,0.644218,0.162239,0.585206,0.200706,0.196121,0.291600,46.231656
5,0.5,0.1,30,500,25,[5000],85,0.656463,0.203173,0.704398,0.197508,0.656463,0.203173,0.604766,0.234437,0.247775,0.329660,30.507156
6,0.5,0.1,30,1000,30,[10000],90,0.772109,0.198446,0.804091,0.178290,0.772109,0.198446,0.737399,0.240343,0.441013,0.371906,58.442945
7,0.5,0.1,50,100,30,[1000],90,0.594694,0.076214,0.629955,0.083670,0.594694,0.076214,0.560218,0.093020,0.060207,0.077986,91.817242


In [82]:
# 임계치 상위 1%, 0.5% 
df_results = pd.DataFrame(all_results)
df_results

,b11,b12,Nodes,T,epoch,dims,threshold,Accuracy_Mean,Accuracy_Std,Precision_Mean,Precision_Std,Recall_Mean,Recall_Std,F1-score_Mean,F1-score_Std,NMI_Mean,NMI_Std,Simulation_Time
0,0.5,0.1,50,100,30,[1000],80,0.563265,0.056329,0.637132,0.088372,0.563265,0.056329,0.492513,0.085378,0.048938,0.065422,38.051792
1,0.5,0.1,50,150,20,[1500],80,0.559592,0.080457,0.635709,0.109964,0.559592,0.080457,0.476419,0.110911,0.059103,0.111309,17.846837
2,0.5,0.1,50,200,25,[2000],80,0.557959,0.077748,0.635245,0.102552,0.557959,0.077748,0.470537,0.112290,0.053006,0.095317,20.919744


In [84]:
# 임계치 상위 1%, 0.5% 
df_results = pd.DataFrame(all_results)
df_results

,b11,b12,Nodes,T,epoch,dims,threshold,Accuracy_Mean,Accuracy_Std,Precision_Mean,Precision_Std,Recall_Mean,Recall_Std,F1-score_Mean,F1-score_Std,NMI_Mean,NMI_Std,Simulation_Time
0,0.5,0.1,30,100,30,[1000],85,0.628571,0.126348,0.705467,0.120918,0.628571,0.126348,0.572851,0.160096,0.151265,0.206291,20.715516
1,0.5,0.1,30,150,25,[1500],80,0.634694,0.145835,0.714368,0.143908,0.634694,0.145835,0.575702,0.180040,0.185057,0.251030,12.467062
2,0.5,0.1,30,200,30,[2000],80,0.606122,0.123933,0.685321,0.130298,0.606122,0.123933,0.542676,0.153866,0.133359,0.219459,13.309476
3,0.5,0.1,30,300,35,[3000],80,0.646939,0.160345,0.735009,0.139007,0.646939,0.160345,0.582068,0.199670,0.208383,0.278762,16.097692


In [89]:
# 임계치 상위 5%, 1% , bs = 16
df_results = pd.DataFrame(all_results)
df_results

,b11,b12,Nodes,T,epoch,dims,threshold,Accuracy_Mean,Accuracy_Std,Precision_Mean,Precision_Std,Recall_Mean,Recall_Std,F1-score_Mean,F1-score_Std,NMI_Mean,NMI_Std,Simulation_Time
0,0.5,0.1,30,100,20,[1000],80,0.638776,0.158128,0.711992,0.153464,0.638776,0.158128,0.592510,0.181857,0.171369,0.199434,14.851835
1,0.5,0.1,30,150,30,[1500],85,0.621769,0.140384,0.700418,0.132265,0.621769,0.140384,0.562092,0.173859,0.152733,0.247137,16.030449
2,0.5,0.1,30,200,30,[2000],80,0.595918,0.122638,0.692568,0.131912,0.595918,0.122638,0.521746,0.156835,0.123997,0.198769,14.582936
3,0.5,0.1,30,300,30,[3000],85,0.655102,0.156572,0.728338,0.147270,0.655102,0.156572,0.600333,0.192048,0.217089,0.277026,15.153339
4,0.5,0.1,30,400,20,[4000],80,0.674830,0.165716,0.768624,0.130441,0.674830,0.165716,0.614596,0.212922,0.253295,0.289133,10.877886
5,0.5,0.1,30,500,35,[5000],85,0.679592,0.178532,0.761836,0.148287,0.679592,0.178532,0.618540,0.227335,0.267752,0.313791,19.539432
6,0.5,0.1,30,1000,25,[10000],80,0.795238,0.188802,0.839237,0.152730,0.795238,0.188802,0.761634,0.234936,0.476845,0.350371,25.137661


In [101]:
# 임계치 상위 5%, 1% , bs = 16
df_results = pd.DataFrame(all_results)
df_results

,b11,b12,Nodes,T,epoch,dims,threshold,Accuracy_Mean,Accuracy_Std,Precision_Mean,Precision_Std,Recall_Mean,Recall_Std,F1-score_Mean,F1-score_Std,NMI_Mean,NMI_Std,Simulation_Time
0,0.5,0.1,30,100,25,[200],90,0.629252,0.153069,0.695337,0.158276,0.629252,0.153069,0.585815,0.172595,0.160024,0.193121,11.149568
1,0.5,0.1,30,150,25,[300],90,0.617007,0.139991,0.686555,0.139692,0.617007,0.139991,0.560645,0.168869,0.150655,0.257975,10.246517
2,0.5,0.1,30,200,35,[800],90,0.614286,0.116058,0.678011,0.123257,0.614286,0.116058,0.567828,0.139747,0.125684,0.206213,13.806960
3,0.5,0.1,30,300,35,[1200],90,0.622449,0.146832,0.705571,0.137991,0.622449,0.146832,0.559403,0.180919,0.164704,0.262997,13.709479
4,0.5,0.1,30,400,30,[800],90,0.627891,0.142595,0.727413,0.131066,0.627891,0.142595,0.559432,0.178931,0.181519,0.259555,12.127910
5,0.5,0.1,30,500,20,[1000],90,0.619728,0.143320,0.700889,0.139027,0.619728,0.143320,0.555686,0.178457,0.158158,0.254631,8.085967
6,0.5,0.1,30,1000,35,[4000],90,0.684354,0.185910,0.741246,0.166928,0.684354,0.185910,0.631823,0.227045,0.276878,0.337260,18.657044


In [108]:
# 임계치 상위 5%, 1% , bs = 32 (bs 지정 여부 확인)
df_results = pd.DataFrame(all_results)
df_results

,b11,b12,Nodes,T,epoch,dims,threshold,Accuracy_Mean,Accuracy_Std,Precision_Mean,Precision_Std,Recall_Mean,Recall_Std,F1-score_Mean,F1-score_Std,NMI_Mean,NMI_Std,Simulation_Time
0,0.5,0.1,30,100,35,[30],90,0.668027,0.132473,0.716270,0.119747,0.668027,0.132473,0.636598,0.162283,0.173597,0.199484,16.198307
1,0.5,0.1,30,150,30,[30],90,0.658503,0.133253,0.729770,0.120920,0.658503,0.133253,0.616381,0.165505,0.185625,0.220307,22.708695
2,0.5,0.1,30,200,35,[30],90,0.620408,0.152799,0.689025,0.159694,0.620408,0.152799,0.572578,0.177871,0.150118,0.205127,17.214694
3,0.5,0.1,30,300,35,[30],90,0.626531,0.168760,0.713167,0.166223,0.626531,0.168760,0.565577,0.203966,0.175316,0.223914,19.372160
4,0.5,0.1,30,400,20,[30],90,0.623810,0.135191,0.713011,0.116669,0.623810,0.135191,0.560978,0.177305,0.138011,0.191924,12.161637
5,0.5,0.1,30,500,25,[30],90,0.610884,0.120541,0.703982,0.124054,0.610884,0.120541,0.546440,0.160548,0.131507,0.172157,14.510808
6,0.5,0.1,30,1000,25,[30],90,0.642857,0.151336,0.746146,0.123484,0.642857,0.151336,0.573497,0.199365,0.191382,0.250284,11.713027
7,0.5,0.1,50,100,25,[50],90,0.552245,0.137429,0.562797,0.141092,0.552245,0.137429,0.536971,0.138970,0.039152,0.081213,25.663569
8,0.5,0.1,50,150,30,[50],90,0.605714,0.109991,0.652761,0.112572,0.605714,0.109991,0.559747,0.136757,0.093921,0.140651,28.387971
9,0.5,0.1,50,200,35,[50],90,0.579592,0.081264,0.635870,0.095487,0.579592,0.081264,0.522122,0.112570,0.060085,0.093582,35.859698


In [111]:
# 임계치 상위 5%, 1% , bs = 10000 
df_results = pd.DataFrame(all_results)
df_results

,b11,b12,Nodes,T,epoch,dims,threshold,Accuracy_Mean,Accuracy_Std,Precision_Mean,Precision_Std,Recall_Mean,Recall_Std,F1-score_Mean,F1-score_Std,NMI_Mean,NMI_Std,Simulation_Time
0,0.5,0.1,30,100,25,[30],90,0.700000,0.129713,0.781890,0.102888,0.700000,0.129713,0.663626,0.158121,0.259648,0.214695,9.289428
1,0.5,0.1,30,150,25,[30],90,0.672109,0.155233,0.726500,0.144120,0.672109,0.155233,0.632414,0.189872,0.214532,0.260281,9.379731
2,0.5,0.1,30,200,35,[30],90,0.614966,0.131460,0.685942,0.127321,0.614966,0.131460,0.559454,0.165174,0.128318,0.213919,14.446048
3,0.5,0.1,30,300,25,[30],90,0.627891,0.137577,0.703399,0.138445,0.627891,0.137577,0.569559,0.175198,0.161348,0.227987,11.393889
4,0.5,0.1,30,400,30,[30],90,0.617687,0.141277,0.745887,0.108617,0.617687,0.141277,0.539885,0.184526,0.161185,0.235652,10.125764
5,0.5,0.1,30,500,35,[30],90,0.677551,0.166379,0.741526,0.145810,0.677551,0.166379,0.628600,0.209664,0.234597,0.281486,15.690382
6,0.5,0.1,30,1000,25,[30],90,0.614286,0.135526,0.730200,0.115630,0.614286,0.135526,0.540422,0.179366,0.145988,0.212528,10.429962
7,0.5,0.1,50,100,35,[50],90,0.568571,0.086378,0.649568,0.116580,0.568571,0.086378,0.491447,0.117924,0.069997,0.105575,15.395844
8,0.5,0.1,50,150,35,[50],90,0.577143,0.073290,0.626447,0.090105,0.577143,0.073290,0.526979,0.101277,0.053183,0.075747,15.198540
9,0.5,0.1,50,200,35,[50],90,0.570204,0.080420,0.654266,0.105340,0.570204,0.080420,0.496110,0.121436,0.058934,0.085139,14.458570


In [21]:
len(y)

2

In [ ]:
import warnings
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from scipy.sparse import diags
from scipy.stats import multivariate_normal
import time 
from sklearn.metrics.pairwise import cosine_similarity
warnings.filterwarnings('ignore')


# ✅ (b11, b12) 조합 리스트
b_combinations = [
    (0.25, 0.05),
    (0.5, 0.1),
    (0.25, 0.01),
    (0.5, 0.01),

]

for b11, b12 in b_combinations:

    all_results = []
    num_simulations = 100

    for n in [30, 50, 100, 150 , 200, 300]:
        # 파라미터 설정 
        k, phi = 2, 0.99
        
        # SBM을 이용한 계수행렬 생성
        sbm_graph = nx.stochastic_block_model([n // k] * k, [[b11 if i == j else b12 for j in range(k)] for i in range(k)], seed=42)
        adj_matrix = nx.to_numpy_array(sbm_graph)
        D_diag = np.sum(adj_matrix, axis=0)  
        D_inv_sqrt = diags(1.0 / np.sqrt(D_diag + 1e-10))
        Phi = phi * D_inv_sqrt @ adj_matrix @ D_inv_sqrt # Φ 생성 

        # 단위근 검정 
        eigenvalues = np.linalg.eigvals(Phi)
        if np.any(np.abs(eigenvalues) >= 1):
            print("⚠ 단위근 존재 (비정상적) - 실행을 중단합니다.")
            sys.exit()  # 실행 중단
        else:
            print("✅ 정상적 (Stationary) - 계속 실행 가능")

        # True label 생성 (test 성능 평가를 위해 쓸 것) 
        True_labels = sum([[i] * (n // k) for i in range(k)], [])
        
        for TS in [100, 150, 200, 300, 400, 500, 1000]:
            accuracy_scores = []
            precision_scores = []
            recall_scores = []
            f1_scores = []
            ari_scores = []
            nmi_scores = []
            train_size = int(TS * 0.8)  # 시뮬레이션 시작 전 train 길이 설정 
            start = time.time() # 시뮬레이션 시작 타임스탬프

            for i in range(1, num_simulations) :
                print(f"{i}th Simulation Start! n = {n}, b11&b12 = {b11}&{b12}, T = {TS} \n")

                # best_ 성능 저장 초기화
                best_acc, best_nmi, best_adj = 0, 0, 0
                best_f1_macro, best_precision_macro, best_recall_macro = 0, 0, 0
                acc_list, results_list, nmi_list, ari_list, f1_macro_list, precision_macro_list, recall_macro_list= [], [], [], [], [], [], []

                np.random.seed(i)                
                Y = np.zeros((TS, n))
                Y[0, :] = np.random.normal(size=n)  
                epsilon = multivariate_normal.rvs(mean=np.zeros(n), cov=np.eye(n), size=TS)
                for t in range(1, TS):
                    Y[t, :] = Phi @ Y[t - 1, :] + epsilon[t, :] # Y[t - 1, :]가 t-1행을 열벡터로 표현하여 계수행렬 Phi와 곱해짐 , 행이 위로 갈수록 과거 

                # Train/Test 분리 (8:2) 후 사이즈 맞추기 
                train_data, test_data = Y[:train_size], Y[train_size:] 
                # train set → 공분산 기반 feature 생성
                cov_matrix_train = np.abs(cosine_similarity(train_data.T))
                np.fill_diagonal(cov_matrix_train, 0)  # self-loop 제거 
                threshold_train = np.percentile(cov_matrix_train, 0)
                precision_matrix = ((cov_matrix_train) > threshold_train).astype(int)

                adj = np.array(np.where(precision_matrix > 0)).T  # A 원소가 1인 순서쌍 출력 ([[1 0] [1 1] ...] 인 array 형태) 
                label = True_labels
                adj = sp.csr_matrix((np.ones(len(adj)), (adj[:,0], adj[:,1])), shape=(len(label), len(label)))


                
                T, F = train_data.shape  
                features = sp.csr_matrix(train_data.T).toarray() # features 차원이 F*T 
                features = torch.FloatTensor(MinMaxScaler().fit_transform(features)) # 행 단위로 정규화 (노드 별 feature vector)
        
                n_nodes, feat_dim = features.shape # 훈련데이터의 차원 정의 (n*F)                
                dims = [feat_dim] + args.dims # feature vector 변환시킬 NN의 레이어 차원 정의 [T , 500]
                layers = args.linlayers 
                
                adj = adj - sp.dia_matrix((adj.diagonal()[np.newaxis, :], [0]), shape=adj.shape)
                adj.eliminate_zeros() # self-loop 제거 후 0이 아닌 값만 남김 
                    
                n = adj.shape[0]    
                
                adj_norm_s = preprocess_graph(adj, args.gnnlayers, norm='sym', renorm=True) # adj 이용해서 스무딩 행렬 H 구함 (3개)
                sm_fea_s = sp.csr_matrix(features).toarray() # feature matrix의 행들을 array 로 배열 
    
                print('Laplacian Smoothing...')
                
                #########기본 셋팅#########
                for a in adj_norm_s: # 위에서 구한 스무딩 행렬을 feature matrix에 곱함
                    sm_fea_s = a.dot(sm_fea_s) #스무딩 완료 (스무딩된 X틸다를 행 별로 나열)
                adj_1st = (adj + sp.eye(n)).toarray() # self-loop 저장한 인접행렬
                Cluster = KMeans(n_clusters=k, random_state=i)
                db, nmi, adjscore, acc, f1_macro, precision_macro, recall_macro = clustering(Cluster, sm_fea_s, True_labels) 
                # sm_fea_s의 각 행을 개체로 클러스터링
                best_cl = db
                adj_label = torch.FloatTensor(adj_1st)

                # 모델 정의
                model = LinTrans(layers, dims) # sm_fea_s에 W를 이용하여 선형변환하는 모델 (layers = 1, dims = [T , 500]) 
                optimizer = optim.Adam(model.parameters(), lr=args.lr) # Adam으로 최적화
                sm_fea_s = torch.FloatTensor(sm_fea_s)
                adj_label = adj_label.reshape([-1,]) #1차원 벡터로 펴줌
                
                if args.cuda:
                    model.cuda()
                    inx = sm_fea_s.cuda() # 스무딩된 행렬 X틸다를 1
                    adj_label = adj_label.cuda()
                pos_num = len(adj.indices) # 인접행렬에서 1인 개수 
                neg_num = n_nodes*n_nodes-pos_num # 인접행렬에서 0인 개수

                # 업데이트할 때마다 움직일 크기
                up_eta = (args.upth_ed - args.upth_st) / (args.epochs/args.upd)  
                low_eta = (args.lowth_ed - args.lowth_st) / (args.epochs/args.upd)                
                
                # update_similarity : sm_fea_s에서 각 행을 정규화하고, 모든 조합을 통해 코사인유사도행렬을 구하여 threshold를 기준으로 pos/neg 라벨링하여 0,1군집의 순서쌍 추출 
                pos_inds, neg_inds = update_similarity(normalize(sm_fea_s.numpy()), args.upth_st, args.lowth_st, pos_num, neg_num) 
                upth, lowth = update_threshold(args.upth_st, args.lowth_st, up_eta, low_eta) # 첫 threshold 정의
                bs = min(args.bs, len(pos_inds)) # pos만큼 배치 길이 지정
                length = len(pos_inds) # pos_inds만큼 length 정의 (샘플 길이)
                pos_inds_cuda = torch.LongTensor(pos_inds).cuda() # pos_inds
                
                print('Start Training...')                             
                for epoch in tqdm(range(args.epochs)):                
                    st, ed = 0, bs 
                    batch_num = 0 
                    model.train() 
                    length = len(pos_inds) 
                
                    while ( ed <= length ): 
                        sampled_neg = torch.LongTensor(np.random.choice(neg_inds, size=ed-st)).cuda() # neg 샘플에서 pos사이즈만큼 샘플링
                        sampled_inds = torch.cat((pos_inds_cuda[st:ed], sampled_neg), 0) #pos와 neg인덱스 이어붙임 
                        optimizer.zero_grad() # 기울기 초기화 
                        # pos/neg 인덱스 (scalar)를 순서쌍 좌표로 만들어 x,y에 저장 (즉, pos/neg 인덱스 value에 담긴 두 노드 찾기)
                        xind = sampled_inds // n_nodes 
                        yind = sampled_inds % n_nodes
                        x = torch.index_select(inx, 0, xind) # sampled_ind에 해당하는 노드의 feature 추출
                        y = torch.index_select(inx, 0, yind) # sampled_ind에 해당하는 노드의 feature 추출
                        zx = model(x) #CNN과 같은 원리로 합성곱 (CNN의 필터링 과정) -> feature vector를 고차원으로 확장하는 단계 
                        zy = model(y) #더욱 고차원의 형태로 feature 표현 
                        batch_label = torch.cat((torch.ones(ed-st), torch.zeros(ed-st))).cuda() # 사전에 정의한 라벨 
                        batch_pred = model.dcs(zx, zy) #확장된 feature vector로 계산한 유사도
                        loss = loss_function(adj_preds=batch_pred, adj_labels=batch_label, n_nodes=ed-st) #확장된 형태의 x,y로 얻어진 유사도가 1/0 분류할 수 있도록 W학습
                        
                        loss.backward()
                        cur_loss = loss.item()
                        optimizer.step()
                        
                        st = ed
                        batch_num += 1
                        if ed < length and ed + bs >= length:
                            ed += length - ed # 다음 배치에 들어올 데이터 크기가 bs크기보다 작으면 나머지만큼만 bs로
                        else:
                            ed += bs # 아니면 다음 배치 사이즈 그대로 유지
                        
                    if (epoch + 1) % args.upd == 0: # 업데이트 횟수 T의 배수번째인 에폭마다 전체 데이터
                        model.eval()
                        mu = model(inx) #전체 feature_matrix의 행을 변환 
                        hidden_emb = mu.cpu().data.numpy()
                        upth, lowth = update_threshold(upth, lowth, up_eta, low_eta) # threshold 업데이트하여 threshold 새로 정의  
                        pos_inds, neg_inds = update_similarity(hidden_emb, upth, lowth, pos_num, neg_num) # 새로 정의된 threshold로 pos neg 샘플 인덱스 (imbalanced)
                        bs = min(args.bs, len(pos_inds)) #bs 새로 정의 - pos_num 새로 정의돼서 
                        pos_inds_cuda = torch.LongTensor(pos_inds).cuda()
                        db, nmi, adjscore, acc, f1_macro, precision_macro, recall_macro = clustering(Cluster, hidden_emb, True_labels) # 지금까지 학습된 모델로 X틸다 확장시킨 행렬로 클러스터링
                        if db >= best_cl:
                            best_cl = db
                            best_acc, best_nmi, best_adj = acc, nmi, adjscore
                            best_f1_macro, best_precision_macro, best_recall_macro = f1_macro, precision_macro, recall_macro
                
                print('모델 성능 평가 단계')
                # ✅ 전체 train set → 공분산 기반 feature 생성
                cov_matrix_train = np.abs(cosine_similarity(test_data.T))
                np.fill_diagonal(cov_matrix_train, 0)  # self-loop 제거 
                threshold_train = np.percentile(cov_matrix_train, 0)
                precision_matrix = ((cov_matrix_train) > threshold_train).astype(int)
                adj = np.array(np.where(precision_matrix > 0)).T  # A 원소가 1인 순서쌍 출력 ([[1 0] [1 1] ...] 인 array 형태) 

                #라벨 리스트를 정수로 변환 
                label = True_labels 
    
                #희소 인접행렬로 변환 (메모리 절약을 위해 GNN에서 필수적)
                adj = sp.csr_matrix((np.ones(len(adj)), (adj[:,0], adj[:,1])), shape=(len(label), len(label)))
                features = sp.csr_matrix(test_data.T).toarray() # features 차원이 n*T
                features = torch.FloatTensor(MinMaxScaler().fit_transform(features))
                        
                # Store original adjacency matrix (without diagonal entries) for later
                adj = adj - sp.dia_matrix((adj.diagonal()[np.newaxis, :], [0]), shape=adj.shape)
                adj.eliminate_zeros()
                    
                n = adj.shape[0]    
                
                adj_norm_s = preprocess_graph(adj, args.gnnlayers, norm='sym', renorm=True) # 여러 개의 정규화된 인접 행렬을 생성 (이게 Ht)
                sm_fea_s = sp.csr_matrix(features).toarray() # feature matrix의 행들을 array 로 배열 
                
                # 스무딩 행렬 만드는 과정 
                for a in adj_norm_s: # 인접행렬을 이용해서 스무딩 행렬 생성함 
                    sm_fea_s = a.dot(sm_fea_s) #스무딩 완료 (a : 스무딩 행렬, sm_fea_s : X틸다) 
                
                # 모델에 input을 위한 test_data 확장 
                test_data_padded = np.zeros((sm_fea_s.shape[0], train_data.shape[0]))  # shape: (n, T_train)
                test_data_padded[:, -sm_fea_s.shape[1]:] = sm_fea_s  # 뒤에 맞춰 붙이기
                sm_fea_s = test_data_padded  # shape: (n, T_train)

                sm_fea_s = torch.FloatTensor(sm_fea_s) 
 
                if args.cuda:
                    model.cuda()
                    inx = sm_fea_s.cuda() # 스무딩된 행렬 X틸다를 행 단위로 array 
                model.eval()
                mu = model(inx)
                hidden_emb = mu.cpu().data.numpy()

                # 예측된 클러스터 레이블
                predict_labels = Cluster.fit_predict(hidden_emb)

                metrics = calculate_clustering_metrics(label, predict_labels)
                print(f"\n✅ 최종 Test 결과 → Evaluations : {metrics}")
                
                # 🔹 지표 저장
                accuracy_scores.append(metrics["Accuracy"])
                recall_scores.append(metrics["Recall"])
                precision_scores.append(metrics["Precision"])
                f1_scores.append(metrics["F1-score"])
                ari_scores.append(metrics["ARI"])
                nmi_scores.append(metrics["NMI"])                


            # 🔹 실행 시간 출력
            end_time = time.time()
            print(f"Execution Time for {b11,b12} - node={n}, T={T}: {end_time - start:.5f} sec")
    
            # 🔹 평균 및 표준 편차 계산
            results_summary = {
                'b11':b11,
                'b12':b12,
                "Nodes": n,
                "T": TS,
                "Accuracy_Mean": np.mean(accuracy_scores),
                "Accuracy_Std": np.std(accuracy_scores),
                "Precision_Mean": np.mean(precision_scores),
                "Precision_Std": np.std(precision_scores),
                "Recall_Mean": np.mean(recall_scores),
                "Recall_Std": np.std(recall_scores),
                "F1-score_Mean": np.mean(f1_scores),
                "F1-score_Std": np.std(f1_scores),
                "ARI_Mean": np.mean(ari_scores),
                "ARI_Std": np.std(ari_scores),
                "NMI_Mean": np.mean(nmi_scores),
                'Simulation_Time' : end_time - start
            }
    



            print( results_summary )
        
            # 🔹 DataFrame에 결과 추가
            all_results.append(results_summary)

# ✅ 모든 결과를 DataFrame으로 변환
df_results = pd.DataFrame(all_results)

# ✅ 최종 CSV 파일로 저장 (모든 결과를 한 개의 파일에 저장)
df_summary.to_csv(f"AGE_test_train_k=0_final.csv", index=False, encoding="utf-8-sig")
    
print("📁 'simulation_results_summary.csv' 파일 저장 완료!")


In [ ]:
import warnings
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from scipy.sparse import diags
from scipy.stats import multivariate_normal
import time 
warnings.filterwarnings('ignore')


# ✅ (b11, b12) 조합 리스트
b_combinations = [
    (0.25, 0.05),
    (0.5, 0.1),
    (0.25, 0.01),
    (0.5, 0.01),

]

for b11, b12 in b_combinations:

    all_results = []
    num_simulations = 100

    for n in [30, 50, 100, 150 , 200, 300]:
        # 파라미터 설정 
        k, phi = 2, 0.99
        
        # SBM을 이용한 계수행렬 생성
        sbm_graph = nx.stochastic_block_model([n // k] * k, [[b11 if i == j else b12 for j in range(k)] for i in range(k)], seed=42)
        adj_matrix = nx.to_numpy_array(sbm_graph)
        D_diag = np.sum(adj_matrix, axis=0)  
        D_inv_sqrt = diags(1.0 / np.sqrt(D_diag + 1e-10))
        Phi = phi * D_inv_sqrt @ adj_matrix @ D_inv_sqrt # Φ 생성 

        # 단위근 검정 
        eigenvalues = np.linalg.eigvals(Phi)
        if np.any(np.abs(eigenvalues) >= 1):
            print("⚠ 단위근 존재 (비정상적) - 실행을 중단합니다.")
            sys.exit()  # 실행 중단
        else:
            print("✅ 정상적 (Stationary) - 계속 실행 가능")

        # True label 생성 (test 성능 평가를 위해 쓸 것) 
        True_labels = sum([[i] * (n // k) for i in range(k)], [])
        
        for TS in [100, 150, 200, 300, 400, 500, 1000]:
            accuracy_scores = []
            precision_scores = []
            recall_scores = []
            f1_scores = []
            ari_scores = []
            nmi_scores = []
            train_size = int(TS * 0.8)  # 시뮬레이션 시작 전 train 길이 설정 
            start = time.time() # 시뮬레이션 시작 타임스탬프

            for i in range(1, num_simulations) :
                print(f"{i}th Simulation Start! n = {n}, b11&b12 = {b11}&{b12}, T = {TS} \n")

                # best_ 성능 저장 초기화
                best_acc, best_nmi, best_adj = 0, 0, 0
                best_f1_macro, best_precision_macro, best_recall_macro = 0, 0, 0
                acc_list, results_list, nmi_list, ari_list, f1_macro_list, precision_macro_list, recall_macro_list= [], [], [], [], [], [], []

                np.random.seed(i)                
                Y = np.zeros((TS, n))
                Y[0, :] = np.random.normal(size=n)  
                epsilon = multivariate_normal.rvs(mean=np.zeros(n), cov=np.eye(n), size=TS)
                for t in range(1, TS):
                    Y[t, :] = Phi @ Y[t - 1, :] + epsilon[t, :] # Y[t - 1, :]가 t-1행을 열벡터로 표현하여 계수행렬 Phi와 곱해짐 , 행이 위로 갈수록 과거 

                # Train/Test 분리 (8:2) 후 사이즈 맞추기 
                train_data, test_data = Y[:train_size], Y[train_size:] 
                # train set → 공분산 기반 feature 생성
                cov_matrix_train = np.abs(np.cov(train_data.T))
                np.fill_diagonal(cov_matrix_train, 0)  # self-loop 제거 
                threshold_train = np.percentile(cov_matrix_train, 90)
                precision_matrix = ((cov_matrix_train) > threshold_train).astype(int)

                adj = np.array(np.where(precision_matrix > 0)).T  # A 원소가 1인 순서쌍 출력 ([[1 0] [1 1] ...] 인 array 형태) 
                label = True_labels
                adj = sp.csr_matrix((np.ones(len(adj)), (adj[:,0], adj[:,1])), shape=(len(label), len(label)))
                T, F = train_data.shape  
                features = sp.csr_matrix(train_data.T).toarray() # features 차원이 F*T 
                features = torch.FloatTensor(MinMaxScaler().fit_transform(features)) # 행 단위로 정규화 (노드 별 feature vector)
        
                n_nodes, feat_dim = features.shape # 훈련데이터의 차원 정의 (n*F)                
                dims = [feat_dim] + args.dims # feature vector 변환시킬 NN의 레이어 차원 정의 [T , 500]
                layers = args.linlayers 
                
                adj = adj - sp.dia_matrix((adj.diagonal()[np.newaxis, :], [0]), shape=adj.shape)
                adj.eliminate_zeros() # self-loop 제거 후 0이 아닌 값만 남김 
                    
                n = adj.shape[0]    
                
                adj_norm_s = preprocess_graph(adj, args.gnnlayers, norm='sym', renorm=True) # adj 이용해서 스무딩 행렬 H 구함 (3개)
                sm_fea_s = sp.csr_matrix(features).toarray() # feature matrix의 행들을 array 로 배열 
    
                print('Laplacian Smoothing...')
                
                #########기본 셋팅#########
                for a in adj_norm_s: # 위에서 구한 스무딩 행렬을 feature matrix에 곱함
                    sm_fea_s = a.dot(sm_fea_s) #스무딩 완료 (스무딩된 X틸다를 행 별로 나열)
                adj_1st = (adj + sp.eye(n)).toarray() # self-loop 저장한 인접행렬
                Cluster = KMeans(n_clusters=k, random_state=i)
                db, nmi, adjscore, acc, f1_macro, precision_macro, recall_macro = clustering(Cluster, sm_fea_s, True_labels) 
                # sm_fea_s의 각 행을 개체로 클러스터링
                best_cl = db
                adj_label = torch.FloatTensor(adj_1st)

                # 모델 정의
                model = LinTrans(layers, dims) # sm_fea_s에 W를 이용하여 선형변환하는 모델 (layers = 1, dims = [T , 500]) 
                optimizer = optim.Adam(model.parameters(), lr=args.lr) # Adam으로 최적화
                sm_fea_s = torch.FloatTensor(sm_fea_s)
                adj_label = adj_label.reshape([-1,]) #1차원 벡터로 펴줌
                
                if args.cuda:
                    model.cuda()
                    inx = sm_fea_s.cuda() # 스무딩된 행렬 X틸다를 1
                    adj_label = adj_label.cuda()
                pos_num = len(adj.indices) # 인접행렬에서 1인 개수 
                neg_num = n_nodes*n_nodes-pos_num # 인접행렬에서 0인 개수

                # 업데이트할 때마다 움직일 크기
                up_eta = (args.upth_ed - args.upth_st) / (args.epochs/args.upd)  
                low_eta = (args.lowth_ed - args.lowth_st) / (args.epochs/args.upd)                
                
                # update_similarity : sm_fea_s에서 각 행을 정규화하고, 모든 조합을 통해 코사인유사도행렬을 구하여 threshold를 기준으로 pos/neg 라벨링하여 0,1군집의 순서쌍 추출 
                pos_inds, neg_inds = update_similarity(normalize(sm_fea_s.numpy()), args.upth_st, args.lowth_st, pos_num, neg_num) 
                upth, lowth = update_threshold(args.upth_st, args.lowth_st, up_eta, low_eta) # 첫 threshold 정의
                bs = min(args.bs, len(pos_inds)) # pos만큼 배치 길이 지정
                length = len(pos_inds) # pos_inds만큼 length 정의 (샘플 길이)
                pos_inds_cuda = torch.LongTensor(pos_inds).cuda() # pos_inds
                
                print('Start Training...')                             
                for epoch in tqdm(range(args.epochs)):                
                    st, ed = 0, bs 
                    batch_num = 0 
                    model.train() 
                    length = len(pos_inds) 
                
                    while ( ed <= length ): 
                        sampled_neg = torch.LongTensor(np.random.choice(neg_inds, size=ed-st)).cuda() # neg 샘플에서 pos사이즈만큼 샘플링
                        sampled_inds = torch.cat((pos_inds_cuda[st:ed], sampled_neg), 0) #pos와 neg인덱스 이어붙임 
                        optimizer.zero_grad() # 기울기 초기화 
                        # pos/neg 인덱스 (scalar)를 순서쌍 좌표로 만들어 x,y에 저장 (즉, pos/neg 인덱스 value에 담긴 두 노드 찾기)
                        xind = sampled_inds // n_nodes 
                        yind = sampled_inds % n_nodes
                        x = torch.index_select(inx, 0, xind) # sampled_ind에 해당하는 노드의 feature 추출
                        y = torch.index_select(inx, 0, yind) # sampled_ind에 해당하는 노드의 feature 추출
                        zx = model(x) #CNN과 같은 원리로 합성곱 (CNN의 필터링 과정) -> feature vector를 고차원으로 확장하는 단계 
                        zy = model(y) #더욱 고차원의 형태로 feature 표현 
                        batch_label = torch.cat((torch.ones(ed-st), torch.zeros(ed-st))).cuda() # 사전에 정의한 라벨 
                        batch_pred = model.dcs(zx, zy) #확장된 feature vector로 계산한 유사도
                        loss = loss_function(adj_preds=batch_pred, adj_labels=batch_label, n_nodes=ed-st) #확장된 형태의 x,y로 얻어진 유사도가 1/0 분류할 수 있도록 W학습
                        
                        loss.backward()
                        cur_loss = loss.item()
                        optimizer.step()
                        
                        st = ed
                        batch_num += 1
                        if ed < length and ed + bs >= length:
                            ed += length - ed # 다음 배치에 들어올 데이터 크기가 bs크기보다 작으면 나머지만큼만 bs로
                        else:
                            ed += bs # 아니면 다음 배치 사이즈 그대로 유지
                        
                    if (epoch + 1) % args.upd == 0: # 업데이트 횟수 T의 배수번째인 에폭마다 전체 데이터
                        model.eval()
                        mu = model(inx) #전체 feature_matrix의 행을 변환 
                        hidden_emb = mu.cpu().data.numpy()
                        upth, lowth = update_threshold(upth, lowth, up_eta, low_eta) # threshold 업데이트하여 threshold 새로 정의  
                        pos_inds, neg_inds = update_similarity(hidden_emb, upth, lowth, pos_num, neg_num) # 새로 정의된 threshold로 pos neg 샘플 인덱스 (imbalanced)
                        bs = min(args.bs, len(pos_inds)) #bs 새로 정의 - pos_num 새로 정의돼서 
                        pos_inds_cuda = torch.LongTensor(pos_inds).cuda()
                        db, nmi, adjscore, acc, f1_macro, precision_macro, recall_macro = clustering(Cluster, hidden_emb, True_labels) # 지금까지 학습된 모델로 X틸다 확장시킨 행렬로 클러스터링
                        if db >= best_cl:
                            best_cl = db
                            best_acc, best_nmi, best_adj = acc, nmi, adjscore
                            best_f1_macro, best_precision_macro, best_recall_macro = f1_macro, precision_macro, recall_macro
                
                print('모델 성능 평가 단계')
                # ✅ 전체 train set → 공분산 기반 feature 생성
                cov_matrix_train = np.abs(np.cov(test_data.T))
                np.fill_diagonal(cov_matrix_train, 0)  # self-loop 제거 
                threshold_train = np.percentile(cov_matrix_train, 90)
                precision_matrix = ((cov_matrix_train) > threshold_train).astype(int)
                adj = np.array(np.where(precision_matrix > 0)).T  # A 원소가 1인 순서쌍 출력 ([[1 0] [1 1] ...] 인 array 형태) 

                #라벨 리스트를 정수로 변환 
                label = True_labels 
    
                #희소 인접행렬로 변환 (메모리 절약을 위해 GNN에서 필수적)
                adj = sp.csr_matrix((np.ones(len(adj)), (adj[:,0], adj[:,1])), shape=(len(label), len(label)))
                features = sp.csr_matrix(test_data.T).toarray() # features 차원이 n*T
                features = torch.FloatTensor(MinMaxScaler().fit_transform(features))
                        
                # Store original adjacency matrix (without diagonal entries) for later
                adj = adj - sp.dia_matrix((adj.diagonal()[np.newaxis, :], [0]), shape=adj.shape)
                adj.eliminate_zeros()
                    
                n = adj.shape[0]    
                
                adj_norm_s = preprocess_graph(adj, args.gnnlayers, norm='sym', renorm=True) # 여러 개의 정규화된 인접 행렬을 생성 (이게 Ht)
                sm_fea_s = sp.csr_matrix(features).toarray() # feature matrix의 행들을 array 로 배열 
                
                # 스무딩 행렬 만드는 과정 
                for a in adj_norm_s: # 인접행렬을 이용해서 스무딩 행렬 생성함 
                    sm_fea_s = a.dot(sm_fea_s) #스무딩 완료 (a : 스무딩 행렬, sm_fea_s : X틸다) 
                
                # 모델에 input을 위한 test_data 확장 
                test_data_padded = np.zeros((sm_fea_s.shape[0], train_data.shape[0]))  # shape: (n, T_train)
                test_data_padded[:, -sm_fea_s.shape[1]:] = sm_fea_s  # 뒤에 맞춰 붙이기
                sm_fea_s = test_data_padded  # shape: (n, T_train)

                sm_fea_s = torch.FloatTensor(sm_fea_s) 
 
                if args.cuda:
                    model.cuda()
                    inx = sm_fea_s.cuda() # 스무딩된 행렬 X틸다를 행 단위로 array 
                model.eval()
                mu = model(inx)
                hidden_emb = mu.cpu().data.numpy()

                # 예측된 클러스터 레이블
                predict_labels = Cluster.fit_predict(hidden_emb)

                metrics = calculate_clustering_metrics(label, predict_labels)
                print(f"\n✅ 최종 Test 결과 → Evaluations : {metrics}")
                
                # 🔹 지표 저장
                accuracy_scores.append(metrics["Accuracy"])
                recall_scores.append(metrics["Recall"])
                precision_scores.append(metrics["Precision"])
                f1_scores.append(metrics["F1-score"])
                ari_scores.append(metrics["ARI"])
                nmi_scores.append(metrics["NMI"])                


            # 🔹 실행 시간 출력
            end_time = time.time()
            print(f"Execution Time for {b11,b12} - node={n}, T={T}: {end_time - start:.5f} sec")
    
            # 🔹 평균 및 표준 편차 계산
            results_summary = {
                'b11':b11,
                'b12':b12,
                "Nodes": n,
                "T": TS,
                "Accuracy_Mean": np.mean(accuracy_scores),
                "Accuracy_Std": np.std(accuracy_scores),
                "Precision_Mean": np.mean(precision_scores),
                "Precision_Std": np.std(precision_scores),
                "Recall_Mean": np.mean(recall_scores),
                "Recall_Std": np.std(recall_scores),
                "F1-score_Mean": np.mean(f1_scores),
                "F1-score_Std": np.std(f1_scores),
                "ARI_Mean": np.mean(ari_scores),
                "ARI_Std": np.std(ari_scores),
                "NMI_Mean": np.mean(nmi_scores),
                "NMI_Std": np.std(nmi_scores),
                'Simulation_Time' : end_time - start
            }
    



            print( results_summary )
        
            # 🔹 DataFrame에 결과 추가
            all_results.append(results_summary)

# ✅ 모든 결과를 DataFrame으로 변환
df_results = pd.DataFrame(all_results)

# ✅ 최종 CSV 파일로 저장 (모든 결과를 한 개의 파일에 저장)
df_summary.to_csv(f"AGE_test_final.csv", index=False, encoding="utf-8-sig")
    
print("📁 'simulation_results_summary.csv' 파일 저장 완료!")


✅ 정상적 (Stationary) - 계속 실행 가능
1th Simulation Start! n = 30, b11&b12 = 0.25&0.05, T = 100 

Laplacian Smoothing...
Start Training...


100%|████████████████████████████████████████| 200/200 [00:01<00:00, 135.11it/s]


모델 성능 평가 단계

✅ 최종 Test 결과 → Evaluations : {'Confusion Matrix': array([[15,  0],
       [14,  1]]), 'Accuracy': 0.5333333333333333, 'Precision': 0.7586206896551724, 'Recall': 0.5333333333333333, 'F1-score': 0.4034090909090909, 'ROC AUC': np.float64(0.5333333333333333), 'ARI': 0.0, 'NMI': np.float64(0.056427881316723556)}
2th Simulation Start! n = 30, b11&b12 = 0.25&0.05, T = 100 

Laplacian Smoothing...
Start Training...


100%|████████████████████████████████████████| 200/200 [00:01<00:00, 156.74it/s]


모델 성능 평가 단계

✅ 최종 Test 결과 → Evaluations : {'Confusion Matrix': array([[ 1, 14],
       [ 0, 15]]), 'Accuracy': 0.5333333333333333, 'Precision': 0.7586206896551724, 'Recall': 0.5333333333333333, 'F1-score': 0.4034090909090909, 'ROC AUC': np.float64(0.5333333333333333), 'ARI': 0.0, 'NMI': np.float64(0.056427881316723556)}
3th Simulation Start! n = 30, b11&b12 = 0.25&0.05, T = 100 

Laplacian Smoothing...
Start Training...


...

모델 성능 평가 단계

✅ 최종 Test 결과 → Evaluations : {'Confusion Matrix': array([[61, 89],
       [56, 94]]), 'Accuracy': 0.5166666666666667, 'Precision': 0.5175143617766569, 'Recall': 0.5166666666666667, 'F1-score': 0.5107467017579377, 'ROC AUC': np.float64(0.5166666666666666), 'ARI': -0.0020781113999530872, 'NMI': np.float64(0.0008575532746611988)}
99th Simulation Start! n = 300, b11&b12 = 0.25&0.05, T = 400 

Laplacian Smoothing...
Start Training...


100%|█████████████████████████████████████████| 200/200 [00:02<00:00, 95.95it/s]


모델 성능 평가 단계

✅ 최종 Test 결과 → Evaluations : {'Confusion Matrix': array([[100,  50],
       [100,  50]]), 'Accuracy': 0.5, 'Precision': 0.5, 'Recall': 0.5, 'F1-score': 0.4857142857142857, 'ROC AUC': np.float64(0.5), 'ARI': -0.002981736861721953, 'NMI': 0.0}
Execution Time for (0.25, 0.05) - node=300, T=320: 360.11041 sec
{'b11': 0.25, 'b12': 0.05, 'Nodes': 300, 'T': 400, 'Accuracy_Mean': np.float64(0.5294612794612794), 'Accuracy_Std': np.float64(0.025996231578354526), 'Precision_Mean': np.float64(0.53340101388863), 'Precision_Std': np.float64(0.029438722816421632), 'Recall_Mean': np.float64(0.5294612794612794), 'Recall_Std': np.float64(0.025996231578354526), 'F1-score_Mean': np.float64(0.5153369248636861), 'F1-score_Std': np.float64(0.028321670969205133), 'ARI_Mean': np.float64(0.0032258340733400262), 'ARI_Std': np.float64(0.010710916636217887), 'NMI_Mean': np.float64(0.005302202920349707), 'NMI_Std': np.float64(0.00925292902961184), 'Simulation_Time': 360.1104142665863}
1th Simulation St

100%|████████████████████████████████████████| 200/200 [00:01<00:00, 106.19it/s]


모델 성능 평가 단계

✅ 최종 Test 결과 → Evaluations : {'Confusion Matrix': array([[ 40, 110],
       [ 36, 114]]), 'Accuracy': 0.5133333333333333, 'Precision': 0.5176221804511278, 'Recall': 0.5133333333333333, 'F1-score': 0.4818039846670768, 'ROC AUC': np.float64(0.5133333333333334), 'ARI': -0.0018240136765260765, 'NMI': np.float64(0.000746704436921344)}
2th Simulation Start! n = 300, b11&b12 = 0.25&0.05, T = 500 



In [ ]:
# ✅ 모든 결과를 DataFrame으로 변환
df_results = pd.DataFrame(all_results)

# ✅ 최종 CSV 파일로 저장 (모든 결과를 한 개의 파일에 저장)
df_summary.to_csv(f"AGE_test_final.csv", index=False, encoding="utf-8-sig")
    
print("📁 'simulation_results_summary.csv' 파일 저장 완료!")


In [ ]:
import warnings
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from scipy.sparse import diags
from scipy.stats import multivariate_normal
import time 
warnings.filterwarnings('ignore')


# ✅ (b11, b12) 조합 리스트
b_combinations = [
    (0.25, 0.01),
    (0.5, 0.01),
    (0.25, 0.05),
    (0.5, 0.1)
]

all_results = []

num_simulations = 100

for b11, b12 in b_combinations:

    for n in [30, 50, 100, 150 , 200, 300]:
        # 파라미터 설정 
        k, phi = 2, 0.99
        
        # SBM을 이용한 계수행렬 생성
        sbm_graph = nx.stochastic_block_model([n // k] * k, [[b11 if i == j else b12 for j in range(k)] for i in range(k)], seed=42)
        adj_matrix = nx.to_numpy_array(sbm_graph)
        D_diag = np.sum(adj_matrix, axis=0)  
        D_inv_sqrt = diags(1.0 / np.sqrt(D_diag + 1e-10))
        Phi = phi * D_inv_sqrt @ adj_matrix @ D_inv_sqrt # Φ 생성 

        # 단위근 검정 
        eigenvalues = np.linalg.eigvals(Phi)
        if np.any(np.abs(eigenvalues) >= 1):
            print("⚠ 단위근 존재 (비정상적) - 실행을 중단합니다.")
            sys.exit()  # 실행 중단
        else:
            print("✅ 정상적 (Stationary) - 계속 실행 가능")

        # True label 생성 (test 성능 평가를 위해 쓸 것) 
        True_labels = sum([[i] * (n // k) for i in range(k)], [])
        
        for TS in [100, 150, 200, 300, 400, 500, 1000]:
            accuracy_scores = []
            precision_scores = []
            f1_scores = []
            ari_scores = []
            nmi_scores = []
            train_size = int(TS * 0.5)  # 시뮬레이션 시작 전 train 길이 설정 
            start = time.time() # 시뮬레이션 시작 타임스탬프

            for i in range(1, num_simulations) :
                print(f"{i}th Simulation Start! n = {n}, b11&b12 = {b11}&{b12}, T = {TS} \n")

                # best_ 성능 저장 초기화
                best_acc, best_nmi, best_adj = 0, 0, 0
                best_f1_macro, best_precision_macro, best_recall_macro = 0, 0, 0
                acc_list, results_list, nmi_list, ari_list, f1_macro_list, precision_macro_list, recall_macro_list= [], [], [], [], [], [], []

                np.random.seed(i)                
                Y = np.zeros((TS, n))
                Y[0, :] = np.random.normal(size=n)  
                epsilon = multivariate_normal.rvs(mean=np.zeros(n), cov=np.eye(n), size=TS)
                for t in range(1, TS):
                    Y[t, :] = Phi @ Y[t - 1, :] + epsilon[t, :]

                # Train/Test 분리 (8:2) 후 사이즈 맞추기 
                train_data, test_data = Y[:train_size], Y[train_size:] 
                # train set → 공분산 기반 feature 생성
                cov_matrix_train = np.abs(np.cov(train_data.T))
                np.fill_diagonal(cov_matrix_train, 0)  # self-loop 제거 
                threshold_train = np.percentile(cov_matrix_train, 90)
                precision_matrix = ((cov_matrix_train) > threshold_train).astype(int)

                adj = np.array(np.where(precision_matrix > 0)).T  # A 원소가 1인 순서쌍 출력 ([[1 0] [1 1] ...] 인 array 형태) 
                label = True_labels
                adj = sp.csr_matrix((np.ones(len(adj)), (adj[:,0], adj[:,1])), shape=(len(label), len(label)))
                T, F = train_data.shape  
                features = sp.csr_matrix(train_data.T).toarray() # features 차원이 F*T 
                features = torch.FloatTensor(MinMaxScaler().fit_transform(features)) # 행 단위로 정규화 (노드 별 feature vector)
        
                n_nodes, feat_dim = features.shape # 훈련데이터의 차원 정의 (n*F)                
                dims = [feat_dim] + args.dims # feature vector 변환시킬 NN의 레이어 차원 정의 [T , 500]
                layers = args.linlayers 
                
                adj = adj - sp.dia_matrix((adj.diagonal()[np.newaxis, :], [0]), shape=adj.shape)
                adj.eliminate_zeros() # self-loop 제거 후 0이 아닌 값만 남김 
                    
                n = adj.shape[0]    
                
                adj_norm_s = preprocess_graph(adj, args.gnnlayers, norm='sym', renorm=True) # adj 이용해서 스무딩 행렬 H 구함 (3개)
                sm_fea_s = sp.csr_matrix(features).toarray() # feature matrix의 행들을 array 로 배열 
    
                print('Laplacian Smoothing...')
                
                #########기본 셋팅#########
                for a in adj_norm_s: # 위에서 구한 스무딩 행렬을 feature matrix에 곱함
                    sm_fea_s = a.dot(sm_fea_s) #스무딩 완료 (스무딩된 X틸다를 행 별로 나열)
                adj_1st = (adj + sp.eye(n)).toarray() # self-loop 저장한 인접행렬
                Cluster = KMeans(n_clusters=k, random_state=i)
                db, nmi, adjscore, acc, f1_macro, precision_macro, recall_macro = clustering(Cluster, sm_fea_s, True_labels) 
                # sm_fea_s의 각 행을 개체로 클러스터링
                best_cl = db
                adj_label = torch.FloatTensor(adj_1st)

                # 모델 정의
                gcn_model = GraphConvolution(input_dim=feat_dim, hidden_dim=64, output_dim=16)
                optimizer = optim.Adam(model.parameters(), lr=args.lr) # Adam으로 최적화
                sm_fea_s = torch.FloatTensor(sm_fea_s)
                adj_label = adj_label.reshape([-1,]) #1차원 벡터로 펴줌
                
                print('Start Training...')                             
                for epoch in tqdm(range(args.epochs)):                
                    st, ed = 0, bs 
                    batch_num = 0 
                    model.train() 
                    length = len(pos_inds) 
                
                    while ( ed <= length ): 
                        sampled_neg = torch.LongTensor(np.random.choice(neg_inds, size=ed-st)).cuda() # neg 샘플에서 pos사이즈만큼 샘플링
                        sampled_inds = torch.cat((pos_inds_cuda[st:ed], sampled_neg), 0) #pos와 neg인덱스 이어붙임 
                        optimizer.zero_grad() # 기울기 초기화 
                        # pos/neg 인덱스 (scalar)를 순서쌍 좌표로 만들어 x,y에 저장 (즉, pos/neg 인덱스 value에 담긴 두 노드 찾기)
                        xind = sampled_inds // n_nodes 
                        yind = sampled_inds % n_nodes
                        x = torch.index_select(inx, 0, xind) # sampled_ind에 해당하는 노드의 feature 추출
                        y = torch.index_select(inx, 0, yind) # sampled_ind에 해당하는 노드의 feature 추출
                        zx = model(x) #CNN과 같은 원리로 합성곱 (CNN의 필터링 과정) -> feature vector를 고차원으로 확장하는 단계 
                        zy = model(y) #더욱 고차원의 형태로 feature 표현 
                        batch_label = torch.cat((torch.ones(ed-st), torch.zeros(ed-st))).cuda() # 사전에 정의한 라벨 
                        batch_pred = model.dcs(zx, zy) #확장된 feature vector로 계산한 유사도
                        loss = loss_function(adj_preds=batch_pred, adj_labels=batch_label, n_nodes=ed-st) #확장된 형태의 x,y로 얻어진 유사도가 1/0 분류할 수 있도록 W학습
                        
                        loss.backward()
                        cur_loss = loss.item()
                        optimizer.step()
                        
                        st = ed
                        batch_num += 1
                        if ed < length and ed + bs >= length:
                            ed += length - ed # 다음 배치에 들어올 데이터 크기가 bs크기보다 작으면 나머지만큼만 bs로
                        else:
                            ed += bs # 아니면 다음 배치 사이즈 그대로 유지
                        
                    if (epoch + 1) % args.upd == 0: # 업데이트 횟수 T의 배수번째인 에폭마다 전체 데이터
                        model.eval()
                        mu = model(inx) #전체 feature_matrix의 행을 변환 
                        hidden_emb = mu.cpu().data.numpy()
                        upth, lowth = update_threshold(upth, lowth, up_eta, low_eta) # threshold 업데이트하여 threshold 새로 정의  
                        pos_inds, neg_inds = update_similarity(hidden_emb, upth, lowth, pos_num, neg_num) # 새로 정의된 threshold로 pos neg 샘플 인덱스 (imbalanced)
                        bs = min(args.bs, len(pos_inds)) #bs 새로 정의 - pos_num 새로 정의돼서 
                        pos_inds_cuda = torch.LongTensor(pos_inds).cuda()
                        db, nmi, adjscore, acc, f1_macro, precision_macro, recall_macro = clustering(Cluster, hidden_emb, True_labels) # 지금까지 학습된 모델로 X틸다 확장시킨 행렬로 클러스터링
                        if db >= best_cl:
                            best_cl = db
                            best_acc, best_nmi, best_adj = acc, nmi, adjscore
                            best_f1_macro, best_precision_macro, best_recall_macro = f1_macro, precision_macro, recall_macro
                
                print('모델 성능 평가 단계')
                # ✅ 전체 train set → 공분산 기반 feature 생성
                cov_matrix_train = np.abs(np.cov(test_data.T))
                np.fill_diagonal(cov_matrix_train, 0)  # self-loop 제거 
                threshold_train = np.percentile(cov_matrix_train, 90)
                precision_matrix = ((cov_matrix_train) > threshold_train).astype(int)
                adj = np.array(np.where(precision_matrix > 0)).T  # A 원소가 1인 순서쌍 출력 ([[1 0] [1 1] ...] 인 array 형태) 

                #라벨 리스트를 정수로 변환 
                label = True_labels 
    
                #희소 인접행렬로 변환 (메모리 절약을 위해 GNN에서 필수적)
                adj = sp.csr_matrix((np.ones(len(adj)), (adj[:,0], adj[:,1])), shape=(len(label), len(label)))
                features = sp.csr_matrix(test_data.T).toarray() # features 차원이 n*T
                features = torch.FloatTensor(MinMaxScaler().fit_transform(features))
                        
                # Store original adjacency matrix (without diagonal entries) for later
                adj = adj - sp.dia_matrix((adj.diagonal()[np.newaxis, :], [0]), shape=adj.shape)
                adj.eliminate_zeros()
                    
                n = adj.shape[0]    
                
                adj_norm_s = preprocess_graph(adj, args.gnnlayers, norm='sym', renorm=True) # 여러 개의 정규화된 인접 행렬을 생성 (이게 Ht)
                sm_fea_s = sp.csr_matrix(features).toarray() # feature matrix의 행들을 array 로 배열 
    
                print('Training data Laplacian Smoothing...')

                # 스무딩 행렬 만드는 과정 
                for a in adj_norm_s: # 인접행렬을 이용해서 스무딩 행렬 생성함 
                    sm_fea_s = a.dot(sm_fea_s) #스무딩 완료 (a : 스무딩 행렬, sm_fea_s : X틸다) 
                
                adj_1st = (adj + sp.eye(n)).toarray() # self-loop 저장한 인접행렬
                #Cluster = SpectralClustering(n_clusters=k, affinity = 'precomputed')
                                
                sm_fea_s = torch.FloatTensor(sm_fea_s) 
                adj_label = adj_label.reshape([-1,])

                
                # 모델에 input을 위한 test_data 확장 
                test_data_padded = np.zeros_like(train_data)
                test_data_padded[-len(test_data):] = test_data
                test_data = test_data_padded

                if train_data.shape == test_data.shape:
                    continue
                else:
                    print('train ,test 형태 다름 ')
                    break 

                if args.cuda:
                    model.cuda()
                    inx = sm_fea_s.cuda() # 스무딩된 행렬 X틸다를 행 단위로 array 
                    adj_label = adj_label.cuda()
                model.eval()
                mu = model(inx)
                hidden_emb = mu.cpu().data.numpy()

                # 예측된 클러스터 레이블
                predict_labels = Cluster.fit_predict(hidden_emb)

                metrics = calculate_clustering_metrics(label, predict_labels)


                print(f"\n✅ 최종 Test 결과 → Evaluations : {metrics}")
    
                # 🔹 지표 저장
                accuracy_scores.append(metrics["Accuracy"])
                precision_scores.append(metrics["Precision"])
                f1_scores.append(metrics["F1-score"])
                ari_scores.append(metrics["ARI"])
                nmi_scores.append(metrics["NMI"])                


        # 🔹 실행 시간 출력
        end_time = time.time()
        print(f"Execution Time for {b11,b12} - node={n}, T={T}: {end_time - start:.5f} sec")

        # 🔹 평균 및 표준 편차 계산
        results_summary = {
            'b11':b11,
            'b12':b12,
            "Nodes": n,
            "T": T,
            "Accuracy_Mean": np.mean(accuracy_scores),
            "Accuracy_Std": np.std(accuracy_scores),
            "Precision_Mean": np.mean(precision_scores),
            "Precision_Std": np.std(precision_scores),
            "F1-score_Mean": np.mean(f1_scores),
            "F1-score_Std": np.std(f1_scores),
            "ARI_Mean": np.mean(ari_scores),
            "ARI_Std": np.std(ari_scores),
            "NMI_Mean": np.mean(nmi_scores),
            "NMI_Std": np.std(nmi_scores),
            'Simulation_Time' : end_time - start
        }

        print( results_summary )
    
        # 🔹 DataFrame에 결과 추가
        all_results.append(results_summary)

# ✅ 모든 결과를 DataFrame으로 변환
df_results = pd.DataFrame(all_results)

# ✅ (b11, b12, n, T) 조합별 평균 및 표준편차 계산
df_summary = df_results.groupby(["b11", "b12", "n", "T"]).agg(["mean", "std"]).reset_index()
df_summary.columns = ["_".join(col).strip() for col in df_summary.columns.values]

# ✅ 최종 CSV 파일로 저장 (모든 결과를 한 개의 파일에 저장)
df_summary.to_csv("AGE_test_final.csv", index=False, encoding="utf-8-sig")

print("📁 'simulation_results_summary.csv' 파일 저장 완료!")

In [ ]:
import warnings
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from scipy.sparse import diags
from scipy.stats import multivariate_normal
from sklearn.covariance import GraphicalLassoCV
warnings.filterwarnings('ignore')


# ✅ (b11, b12) 조합 리스트
b_combinations = [
    (0.25, 0.01),
    (0.5, 0.01),
    (0.25, 0.05),
    (0.5, 0.1)
]

all_results = []


num_simulations = 100
all_results_list = []  # ✅ 전체 결과 저장 리스트

for b11, b12 in b_combinations:

    for n in [ 30, 50, 100, 150 ]:
        # 파라미터 설정 
        k, phi = 2, 0.99
        
        # SBM 그래프 생성
        sbm_graph = nx.stochastic_block_model([n // k] * k, [[b11 if i == j else b12 for j in range(k)] for i in range(k)], seed=42)
        adj_matrix = nx.to_numpy_array(sbm_graph)
        # D 행렬 및 D^{-1/2} 계산
        D_diag = np.sum(adj_matrix, axis=0)  # 입차수 (undirected이므로 동일)
        D_inv_sqrt = diags(1.0 / np.sqrt(D_diag + 1e-10))
        # Phi 행렬 계산
        Phi = phi * D_inv_sqrt @ adj_matrix @ D_inv_sqrt
        eigenvalues = np.linalg.eigvals(Phi)

        # 단위근 여부 확인 후 실행 중단
        if np.any(np.abs(eigenvalues) >= 1):
            print("⚠ 단위근 존재 (비정상적) - 실행을 중단합니다.")
            sys.exit()  # 실행 중단
        else:
            print("✅ 정상적 (Stationary) - 계속 실행 가능")
            
        True_labels = sum([[i] * (n // k) for i in range(k)], [])
        
        # 결과 출력
        for TS in [100, 150, 200, 300, 400, 500, 1000]:

            # 🔹 평가 지표 저장을 위한 리스트 초기화
            accuracy_scores = []
            precision_scores = []
            recall_scores = []
            f1_scores = []
            roc_auc_scores = []
            ari_scores = []
            nmi_scores = []
            total_confusion_matrix = None  
            confusion_matrices = []
            train_size = int(TS * 0.8)  # Train size 80% of T
            # ✅ best_ 변수 초기화
            best_acc, best_nmi, best_adj = 0, 0, 0
            best_f1_macro, best_precision_macro, best_recall_macro = 0, 0, 0
            acc_list, results_list, nmi_list, ari_list, f1_macro_list, precision_macro_list, recall_macro_list= [], [], [], [], [], [], []


            start = time.time()

            for i in range(1, num_simulations) :
                print(f"{i}th Simulation Start! |n")
                np.random.seed(i)
                
                # 4️⃣ VAR 모델 데이터 생성
                Y = np.zeros((TS, n))
                Y[0, :] = np.random.normal(size=n)  
                epsilon = multivariate_normal.rvs(mean=np.zeros(n), cov=np.eye(n), size=TS)
                for t in range(1, TS):
                    Y[t, :] = Phi @ Y[t - 1, :] + epsilon[t, :]


                ######DGP 생성#######
                # 💡 1. Train/Test 분리 (8:2)
                print(train_size)
                train_data, test_data = Y[:train_size], Y[train_size:] # 80*30 , 20*30                
                print("📈 전체 train set으로 학습 후 test set 평가 시작")
            
#                # ✅ 전체 train set → 공분산 기반 feature 생성
#                cov_matrix_train = np.abs(np.cov(train_data.T))
#                np.fill_diagonal(cov_matrix_train, 0)  # self-loop 제거 
#                threshold_train = np.percentile(cov_matrix_train, 90)
#                precision_matrix = ((cov_matrix_train) > threshold_train).astype(int)
#                adj = np.array(np.where(precision_matrix > 0)).T  # A 원소가 1인 순서쌍 출력 ([[1 0] [1 1] ...] 인 array 형태) 

                #라벨 리스트를 정수로 변환 
                label = True_labels 
    
                #희소 인접행렬로 변환 (메모리 절약을 위해 GNN에서 필수적)
#                adj = sp.csr_matrix((np.ones(len(adj)), (adj[:,0], adj[:,1])), shape=(len(label), len(label)))
                T, F = train_data.shape  # (T=500*0.8, F=50)
                features = sp.csr_matrix(train_data.T).toarray() # features 차원이 n*T
                features = torch.FloatTensor(MinMaxScaler().fit_transform(features))
        
                n_nodes, feat_dim = features.shape # 훈련데이터의 차원 정의 (n*F)
                
                dims = [feat_dim] + args.dims # 레이어 차원이 [T 500]
                layers = args.linlayers # model에 필요한 파라미터 정의 =1
                
                # Store original adjacency matrix (without diagonal entries) for later
#                adj = adj - sp.dia_matrix((adj.diagonal()[np.newaxis, :], [0]), shape=adj.shape)
#                adj.eliminate_zeros()
                    
                n = adj.shape[0]    
                
#                adj_norm_s = preprocess_graph(adj, args.gnnlayers, norm='sym', renorm=True) # 여러 개의 정규화된 인접 행렬을 생성 (이게 Ht)
                sm_fea_s = sp.csr_matrix(features).toarray() # feature matrix의 행들을 array 로 배열 
    
                print('Laplacian Smoothing...')

                # 스무딩 행렬 만드는 과정 
#                for a in adj_norm_s: # 인접행렬을 이용해서 스무딩 행렬 생성함 
#                    sm_fea_s = a.dot(sm_fea_s) #스무딩 완료 (a : 스무딩 행렬, sm_fea_s : X틸다) 
                
                adj_1st = (adj + sp.eye(n)).toarray() # self-loop 저장한 인접행렬
                #Cluster = SpectralClustering(n_clusters=k, affinity = 'precomputed')
                
                Cluster = KMeans(n_clusters=k, random_state=i)
                db, nmi, adjscore, acc, f1_macro, precision_macro, recall_macro = clustering(Cluster, sm_fea_s, True_labels) #sm_fea_s의 행 단위로 spectral clustering
                best_cl = db
                adj_label = torch.FloatTensor(adj_1st)

                # 모델 정의 
                model = LinTrans(layers, dims) # W 곱함 (layers = 1, dims = [500]) 
                optimizer = optim.Adam(model.parameters(), lr=args.lr) # Adam으로 최적화 
                
                sm_fea_s = torch.FloatTensor(sm_fea_s) 
                adj_label = adj_label.reshape([-1,])
                if args.cuda:
                    model.cuda()
                    inx = sm_fea_s.cuda() # 스무딩된 행렬 X틸다를 행 단위로 array , inx는 30*80 (n*T)
                    adj_label = adj_label.cuda()
                pos_num = len(adj.indices) # 인접행렬에서 1인 개수 
                neg_num = n_nodes*n_nodes-pos_num # 인접행렬에서 0인 개수

                # 업데이트할 때마다 움직일 크기
                up_eta = (args.upth_ed - args.upth_st) / (args.epochs/args.upd)  
                low_eta = (args.lowth_ed - args.lowth_st) / (args.epochs/args.upd)
                
                
                # threshold로 pos/neg_num 정의한 후 스무딩된 행렬의 유사도 행렬을 평탄화(2D -> 1D)해서 pos/neg의 인덱스 추출 
                pos_inds, neg_inds = update_similarity(normalize(sm_fea_s.numpy()), args.upth_st, args.lowth_st, pos_num, neg_num) # pos_num, neg_num 무시
                upth, lowth = update_threshold(args.upth_st, args.lowth_st, up_eta, low_eta) # 업데이트 된 threshold 
                
                bs = min(args.bs, len(pos_inds)) # pos만큼 길이 지정
                length = len(pos_inds) # pos_inds만큼 length 정의 (샘플 길이)
                
                pos_inds_cuda = torch.LongTensor(pos_inds).cuda() # pos_inds 
                
                print('Start Training...')                             
                for epoch in tqdm(range(args.epochs)):                
                    st, ed = 0, bs 
                    batch_num = 0 
                    model.train() 
                    length = len(pos_inds) 
                
                    while ( ed <= length ): 
                        sampled_neg = torch.LongTensor(np.random.choice(neg_inds, size=ed-st)).cuda() # neg인 인덱스집합에서 pos사이즈만큼 랜덤 샘플링
                        sampled_inds = torch.cat((pos_inds_cuda[st:ed], sampled_neg), 0) #pos와 neg 인덱스 이어붙임 (유사도행렬에서 얻어진 인덱스 ) 
                        #t = time.time()
                        optimizer.zero_grad()
                        # 샘플링된 유사도 값들을(1D) 다시 matrix로 만드는 과정(2D) 
                        xind = sampled_inds // n_nodes 
                        yind = sampled_inds % n_nodes
                        x = torch.index_select(inx, 0, xind) # 행 기준으로 xind 인덱스인 행 추출하겠다 , x는 개수 * T
                        y = torch.index_select(inx, 0, yind) # sampled_inds의 스칼라 좌표를 쌍으로 만들어서 (value -> (x,y)) 2차원에서의 좌표를 다시 찾음  
                        zx = model(x) #CNN과 같은 원리로 합성곱 (CNN의 필터링 과정) 과정을 거쳐 
                        zy = model(y) #더욱 고차원의 형태로 feature 표현 
                        batch_label = torch.cat((torch.ones(ed-st), torch.zeros(ed-st))).cuda() # sample의 라벨 (인접행렬 원소)
                        batch_pred = model.dcs(zx, zy) #유사도 계산 
                        loss = loss_function(adj_preds=batch_pred, adj_labels=batch_label, n_nodes=ed-st) #즉, NN를 통해 확장된 형태의 x,y로 얻어진 유사도가 true label을 잘 따라갈 수 있도록 W학습 
                        
                        loss.backward()
                        cur_loss = loss.item()
                        optimizer.step()
                        
                        st = ed 
                        batch_num += 1
                        if ed < length and ed + bs >= length:
                            ed += length - ed # 다음 배치가 bs크기보다 작으면 나머지만큼만 bs로
                        else:
                            ed += bs # 아니면 다음 배치 사이즈 그대로 유지 
                            
                            
                    if (epoch + 1) % args.upd == 0: # 업데이트 횟수T의 배수번째인 에폭마다 전체 데이터
                        model.eval()
                        mu = model(inx) 
                        hidden_emb = mu.cpu().data.numpy()

                        
                        upth, lowth = update_threshold(upth, lowth, up_eta, low_eta) # threshold 업데이트하여 pos, neg_num 새로 정의  
                        pos_inds, neg_inds = update_similarity(hidden_emb, upth, lowth, pos_num, neg_num) # 새로 정의된 pos neg 샘플 인덱스
                        bs = min(args.bs, len(pos_inds)) 
                        pos_inds_cuda = torch.LongTensor(pos_inds).cuda()
                            
                        #tqdm.write("Epoch: {}, train_loss_gae={:.5f}, time={:.5f}".format(
                        #    epoch + 1, cur_loss, time.time() - t))
                            
                        db, nmi, adjscore, acc, f1_macro, precision_macro, recall_macro = clustering(Cluster, hidden_emb, True_labels) # 지금까지 학습된 모델로 출력한 유사도행렬로 클러스터링 
                            
                        if db >= best_cl:
                            best_cl = db
                            best_acc, best_nmi, best_adj = acc, nmi, adjscore
                            best_f1_macro, best_precision_macro, best_recall_macro = f1_macro, precision_macro, recall_macro

                
                print('모델 성능 평가 단계')
                # ✅ 전체 train set → 공분산 기반 feature 생성
                cov_matrix_train = np.abs(np.cov(test_data.T))
                np.fill_diagonal(cov_matrix_train, 0)  # self-loop 제거 
                threshold_train = np.percentile(cov_matrix_train, 90)
                precision_matrix = ((cov_matrix_train) > threshold_train).astype(int)
                adj = np.array(np.where(precision_matrix > 0)).T  # A 원소가 1인 순서쌍 출력 ([[1 0] [1 1] ...] 인 array 형태) 

                #라벨 리스트를 정수로 변환 
                label = True_labels 
    
                #희소 인접행렬로 변환 (메모리 절약을 위해 GNN에서 필수적)
                adj = sp.csr_matrix((np.ones(len(adj)), (adj[:,0], adj[:,1])), shape=(len(label), len(label)))
                T, F = test_data.shape  # (T=500*0.8, F=50)
                features = sp.csr_matrix(test_data.T).toarray() # features 차원이 n*T
                features = torch.FloatTensor(MinMaxScaler().fit_transform(features))
                        
                dims = [feat_dim] + args.dims # 레이어 차원이 [T 500]
                layers = args.linlayers # model에 필요한 파라미터 정의 =1
                
                # Store original adjacency matrix (without diagonal entries) for later
                adj = adj - sp.dia_matrix((adj.diagonal()[np.newaxis, :], [0]), shape=adj.shape)
                adj.eliminate_zeros()
                    
                n = adj.shape[0]    
                
                adj_norm_s = preprocess_graph(adj, args.gnnlayers, norm='sym', renorm=True) # 여러 개의 정규화된 인접 행렬을 생성 (이게 Ht)
                sm_fea_s = sp.csr_matrix(features).toarray() # feature matrix의 행들을 array 로 배열 
    
                print('Laplacian Smoothing...')

                # 스무딩 행렬 만드는 과정 
                for a in adj_norm_s: # 인접행렬을 이용해서 스무딩 행렬 생성함 
                    sm_fea_s = a.dot(sm_fea_s) #스무딩 완료 (a : 스무딩 행렬, sm_fea_s : X틸다) 
                
                adj_1st = (adj + sp.eye(n)).toarray() # self-loop 저장한 인접행렬
                #Cluster = SpectralClustering(n_clusters=k, affinity = 'precomputed')
                                
                sm_fea_s = torch.FloatTensor(sm_fea_s) 
                adj_label = adj_label.reshape([-1,])

                if args.cuda:
                    model.cuda()
                    inx = sm_fea_s.cuda() # 스무딩된 행렬 X틸다를 행 단위로 array 
                    adj_label = adj_label.cuda()
                model.eval()
                mu = model(inx)
                hidden_emb = mu.cpu().data.numpy()

                # 예측된 클러스터 레이블
                predict_labels = Cluster.fit_predict(hidden_emb)

                metrics = calculate_clustering_metrics(label, predict_labels)

        

                print(f"\n✅ 최종 Test 결과 → Evaluations : {metrics}")
    
                # 🔹 지표 저장
                accuracy_scores.append(metrics["Accuracy"])
                precision_scores.append(metrics["Precision"])
                f1_scores.append(metrics["F1-score"])
                ari_scores.append(metrics["ARI"])
                nmi_scores.append(metrics["NMI"])                


        # 🔹 실행 시간 출력
        end_time = time.time()
        print(f"Execution Time for {b11,b12} - node={n}, T={T}: {end_time - start:.5f} sec")

        # 🔹 평균 및 표준 편차 계산
        results_summary = {
            'b11':b11,
            'b12':b12,
            "Nodes": n,
            "T": T,
            "Accuracy_Mean": np.mean(accuracy_scores),
            "Accuracy_Std": np.std(accuracy_scores),
            "Precision_Mean": np.mean(precision_scores),
            "Precision_Std": np.std(precision_scores),
            "F1-score_Mean": np.mean(f1_scores),
            "F1-score_Std": np.std(f1_scores),
            "ARI_Mean": np.mean(ari_scores),
            "ARI_Std": np.std(ari_scores),
            "NMI_Mean": np.mean(nmi_scores),
            "NMI_Std": np.std(nmi_scores),
            'Simulation_Time' : end_time - start
        }


        print( results_summary )
    
        # 🔹 DataFrame에 결과 추가
        all_results.append(results_summary)

# ✅ 모든 결과를 DataFrame으로 변환
df_results = pd.DataFrame(all_results_list)

# ✅ (b11, b12, n, T) 조합별 평균 및 표준편차 계산
df_summary = df_results.groupby(["b11", "b12", "n", "T"]).agg(["mean", "std"]).reset_index()
df_summary.columns = ["_".join(col).strip() for col in df_summary.columns.values]

# ✅ 최종 CSV 파일로 저장 (모든 결과를 한 개의 파일에 저장)
df_summary.to_csv("AGE_KMeans_results_summary.csv", index=False, encoding="utf-8-sig")

print("📁 'simulation_results_summary.csv' 파일 저장 완료!")

In [ ]:
import warnings
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from scipy.sparse import diags
from scipy.stats import multivariate_normal
from sklearn.covariance import GraphicalLassoCV
from sklearn.model_selection import train_test_split
import torch
import torch.optim as optim
import scipy.sparse as sp
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from tqdm import tqdm

warnings.filterwarnings('ignore')

num_simulations = 100
all_results_list = []  # ✅ 전체 결과 저장 리스트

for n in [30, 50, 100, 200]:
    # 파라미터 설정 
    k, b11, b12, phi = 2, 0.25, 0.01, 0.99
    
    # SBM 그래프 생성
    sbm_graph = nx.stochastic_block_model([n // k] * k, [[b11 if i == j else b12 for j in range(k)] for i in range(k)], seed=42)
    adj_matrix = nx.to_numpy_array(sbm_graph)
    print(f"T = 생성된 인접행렬: ")
    print(adj_matrix ,'\n')
    
    # D 행렬 및 D^{-1/2} 계산
    D_diag = np.sum(adj_matrix, axis=0)  # 입차수 (undirected이므로 동일)
    D_inv_sqrt = diags(1.0 / np.sqrt(D_diag + 1e-10))
    
    # Phi 행렬 계산
    Phi = phi * D_inv_sqrt @ adj_matrix @ D_inv_sqrt
    True_labels = sum([[i] * (n // k) for i in range(k)], [])
    
    # 결과 출력
    print("Phi 행렬:\n", Phi)
    print("True Labels:", True_labels)
    for T in [10, 50, 100, 150, 200, 300, 400, 500, 1000]:
        # ✅ best_ 변수 초기화
        best_acc, best_nmi, best_adj = 0, 0, 0
        best_f1_macro, best_precision_macro, best_recall_macro = 0, 0, 0
        best_f1_micro, best_precision_micro, best_recall_micro = 0, 0, 0
        acc_list, results_list, nmi_list, ari_list, f1_macro_list, precision_macro_list, recall_macro_list, f1_micro_list, precision_micro_list, recall_micro_list = [], [], [], [], [], [], [], [], [], []

        for i in range(1, num_simulations) :
            print(f"{i}th Simulation Start! |n")
            np.random.seed(i)

            # 4️⃣ VAR 모델 데이터 생성
            Y = np.zeros((T, n))
            Y[0, :] = np.random.normal(size=n)  
            epsilon = multivariate_normal.rvs(mean=np.zeros(n), cov=np.eye(n), size=T)
            for t in range(1, T):
                Y[t, :] = Phi @ Y[t - 1, :] + epsilon[t, :]
            
            glasso = GraphicalLassoCV(cv=5).fit(Y)
            best_alpha, precision_matrix = glasso.alpha_, glasso.precision_
            np.fill_diagonal(precision_matrix, 0)
            precision_matrix = (np.abs(precision_matrix) > np.percentile(np.abs(precision_matrix), 90)).astype(int)
            
            adj = np.array(np.where(precision_matrix > 0)).T  # 값이 0이 아닌 (i, j) 위치 찾기
            adj = np.array(adj)
            adj = np.vstack((adj, adj[:,[1,0]]))
            adj = np.unique(adj, axis=0) 


            #라벨 리스트를 정수로 변환 
            label = True_labels 
            labelset = np.unique(label)
            labeldict = dict(zip(labelset, range(len(labelset))))
            label = np.array([labeldict[x] for x in label])


            #희소 인접행렬로 변환 (메모리 절약을 위해 GNN에서 필수적)
            adj = sp.csr_matrix((np.ones(len(adj)), (adj[:,0], adj[:,1])), shape=(len(label), len(label)))
            print("🎯 인접행렬 희소 행렬 변환 완료:", adj.shape) 


            T, F = Y.shape  # (T=500, F=50)

            # ✅ 특징 행렬 생성 (노드=50, 특징=500)
            features = sp.csr_matrix(Y.T).toarray()  
            features = torch.FloatTensor(MinMaxScaler().fit_transform(features))
            
            print("🎯 PyTorch Tensor 변환 완료:", features.shape)  # (50, 500)

            # Train-Test Split (80:20)
            train_data, test_data = train_test_split(features.numpy(), test_size=0.2, random_state=42)
            train_label, test_label = train_test_split(label, test_size=0.2, random_state=42)

            # k_ 설정
            def decompose(adj, dataset, norm='sym', renorm=True):
                adj = sp.coo_matrix(adj)
                ident = sp.eye(adj.shape[0])
                if renorm:
                    adj_ = adj + ident
                else:
                    adj_ = adj
            
                rowsum = np.array(adj_.sum(1))
                
                if norm == 'sym':
                    degree_mat_inv_sqrt = sp.diags(np.power(rowsum, -0.5).flatten())
                    adj_normalized = adj_.dot(degree_mat_inv_sqrt).transpose().dot(degree_mat_inv_sqrt).tocoo()
                    laplacian = ident - adj_normalized
                evalue, evector = np.linalg.eig(laplacian.toarray())
                np.save(dataset + ".npy", evalue)
                return (max(evalue)) #최대 고유값 
            k_ = 1 / decompose(precision_matrix, 'Y') 
            print(f'k값 : {k_}')

            n_nodes, feat_dim = features.shape
            dims = [feat_dim] + args.dims


            layers = args.linlayers
            # Store original adjacency matrix (without diagonal entries) for later
                
            adj = adj - sp.dia_matrix((adj.diagonal()[np.newaxis, :], [0]), shape=adj.shape)
            adj.eliminate_zeros()
                
            n = adj.shape[0]

            adj_norm_s = preprocess_graph(adj, args.gnnlayers, norm='sym', renorm=True) # 여러 개의 정규화된 인접 행렬을 생성 (이게 Ht)
            sm_fea_s = sp.csr_matrix(features).toarray() # smoothing을 위한 X 희소 행렬화를 다시 밀집행렬로 (toarray)

            #GNN 학습 준비 
            print('Laplacian Smoothing...')
            for a in adj_norm_s:
                sm_fea_s = a.dot(sm_fea_s)
            adj_1st = (adj + sp.eye(n)).toarray() # self-loop 저장한 인접행렬
            Cluster = KMeans(n_clusters=k)
            db, nmi, adjscore, acc, f1_macro, precision_macro, recall_macro   = clustering(Cluster, sm_fea_s, label) # k-means로 클러스터링
            best_cl = db
            adj_label = torch.FloatTensor(adj_1st)
            model = LinTrans(layers, dims) # GNN 모델 정의 (계층 수(1), 각 레이어 차원 수 (500))
            optimizer = optim.Adam(model.parameters(), lr=args.lr) # Adam으로 최적화 
                    
            sm_fea_s = torch.FloatTensor(sm_fea_s)
            adj_label = adj_label.reshape([-1,])
            if args.cuda:
                model.cuda()
                inx = sm_fea_s.cuda()
                adj_label = adj_label.cuda()

            # similarity matrix(S) 만들기 위해 순위를 이용한 traning sample selection 과정 
            # 초기 S는 코사인 유사도 행렬 
            pos_num = len(adj.indices) 
            neg_num = n_nodes*n_nodes-pos_num
            
            up_eta = (args.upth_ed - args.upth_st) / (args.epochs/args.upd)
            low_eta = (args.lowth_ed - args.lowth_st) / (args.epochs/args.upd)
            
            pos_inds, neg_inds = update_similarity(normalize(sm_fea_s.numpy()), args.upth_st, args.lowth_st, pos_num, neg_num)
            upth, lowth = update_threshold(args.upth_st, args.lowth_st, up_eta, low_eta)
            
            bs = min(args.bs, len(pos_inds))
            length = len(pos_inds)
                
            pos_inds_cuda = torch.LongTensor(pos_inds).cuda()
            print('Start Training...')
            
            #GNN을 이용한 클러스터링 및 학습 과정            
                        
            for epoch in tqdm(range(args.epochs)):
            
                st, ed = 0, bs
                batch_num = 0
                model.train()
                length = len(pos_inds)
            
                while ( ed <= length ):
                    sampled_neg = torch.LongTensor(np.random.choice(neg_inds, size=ed-st)).cuda()
                    sampled_inds = torch.cat((pos_inds_cuda[st:ed], sampled_neg), 0)
                    #t = time.time()
                    optimizer.zero_grad()
                    xind = sampled_inds // n_nodes
                    yind = sampled_inds % n_nodes
                    x = torch.index_select(inx, 0, xind)
                    y = torch.index_select(inx, 0, yind)
                    zx = model(x)
                    zy = model(y)
                    batch_label = torch.cat((torch.ones(ed-st), torch.zeros(ed-st))).cuda()
                    batch_pred = model.dcs(zx, zy)
                    loss = loss_function(adj_preds=batch_pred, adj_labels=batch_label, n_nodes=ed-st)
                    
                    loss.backward()
                    cur_loss = loss.item()
                    optimizer.step()
                    
                    st = ed
                    batch_num += 1
                    if ed < length and ed + bs >= length:
                        ed += length - ed
                    else:
                        ed += bs
                
                        
                if (epoch + 1) % args.upd == 0: #max_iter/T 나머지 == 0 
                    model.eval()
                    mu = model(inx)
                    hidden_emb = mu.cpu().data.numpy()
                    upth, lowth = update_threshold(upth, lowth, up_eta, low_eta) # threshold 업데이트 
                    pos_inds, neg_inds = update_similarity(hidden_emb, upth, lowth, pos_num, neg_num) # S 업데이트 
                    bs = min(args.bs, len(pos_inds))
                    pos_inds_cuda = torch.LongTensor(pos_inds).cuda()
                        
                    #tqdm.write("Epoch: {}, train_loss_gae={:.5f}, time={:.5f}".format(
                    #    epoch + 1, cur_loss, time.time() - t))
                        
                    db, nmi, adjscore, acc, f1_macro, precision_macro, recall_macro   = clustering(Cluster, hidden_emb, label) # k-means로 클러스터링
                        
                    # 최적 성능 업데이트
                    if db >= best_cl:
                        best_cl = db
                        best_acc, best_nmi, best_adj = acc, nmi, adjscore
                        best_f1_macro, best_precision_macro, best_recall_macro = f1_macro, precision_macro, recall_macro
            # 성능 저장
            results_list.append({
                    "n": n,
                    "T": T,
                    "Simulation": i,
                    "Accuracy": best_acc,
                    "NMI": best_nmi,
                    "ARI": best_adj,
                    "F1 Macro": best_f1_macro,
                    "Precision Macro": best_precision_macro,
                    "Recall Macro": best_recall_macro,
                    "F1 Micro": best_f1_micro,
                    "Precision Micro": best_precision_micro,
                    "Recall Micro": best_recall_micro,
                    })                    
            tqdm.write("Optimization Finished!")
            tqdm.write('best_acc: {}, best_nmi: {}, best_adj: {}'.format(best_acc, best_nmi, best_adj))
        
        all_results_list.extend(results_list)

# ✅ 전체 결과 리스트를 DataFrame으로 변환
df_results = pd.DataFrame(all_results_list)

# ✅ (n, T) 조합별 평균 및 표준편차 계산
df_summary = df_results.groupby(["n", "T"]).agg(["mean", "std"]).reset_index()

# ✅ 컬럼명 변환 ("컬럼명_통계량" 형식)
df_summary.columns = ["_".join(col).strip() for col in df_summary.columns.values]

# ✅ CSV 파일로 저장 (하나의 파일에 모든 결과 저장)
df_summary.to_csv("simulation_results_summary.csv", index=False, encoding="utf-8-sig")

print("📁 'simulation_results_summary.csv' 파일 저장 완료!")


In [25]:
best_embed

NameError: name 'best_embed' is not defined